In [1]:
# -*- coding: utf-8 -*-
# ==============================================================================
# CÓDIGO COMBINADO Y CORREGIDO (Celda 0 + Celda 1)
# TFG - Evaluación Impacto EDL en Seguridad Hídrica Salar de Uyuni
# ------------------------------------------------------------------------------
# VERSIÓN MODIFICADA:
# - Usa ode_B_modK para Biodiversidad (modifica K_eff)
# - VALOR BASE de C_max_stress FIJADO a 150.0 en la definición de parámetros
# ==============================================================================

# Mensaje inicial para indicar qué versión del script se está ejecutando.
print("--- Ejecutando Código Combinado (Setup Global + Simulación Interactiva) ---")
# Especifica características clave de esta versión del modelo.
print("--- VERSIÓN: ode_B_modK para Biodiversidad | C_max_stress Base = 150.0 ---")

# --- Importaciones de Bibliotecas ---
# Estas son las herramientas (librerías) que el programa necesita para funcionar.

# Para crear elementos interactivos (sliders, botones, etc.) en la interfaz de usuario.
import ipywidgets as widgets

# Para controlar cómo se muestra la información en el entorno Jupyter (como este notebook).
# display: para mostrar widgets y otros elementos.
# clear_output: para limpiar la salida de una celda antes de mostrar nueva información.
# HTML: para renderizar código HTML directamente.
from IPython.display import display, clear_output, HTML as display_HTML

# Para trabajar con datos en forma de tablas (DataFrames), muy útil para organizar parámetros y resultados.
import pandas as pd

# Para cálculos numéricos eficientes, especialmente con arrays (listas de números) y operaciones matemáticas.
import numpy as np

# Para crear gráficos y visualizaciones.
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec # Para organizar múltiples gráficos en una figura.
from matplotlib.ticker import MaxNLocator

# De la biblioteca SciPy, importamos 'solve_ivp', una función poderosa para resolver
# sistemas de Ecuaciones Diferenciales Ordinarias (EDOs), que son el corazón de este modelo dinámico.
from scipy.integrate import solve_ivp

# Módulo estándar de Python para funciones matemáticas básicas (seno, coseno, pi, etc.).
import math

# Para obtener información detallada cuando ocurren errores, lo que ayuda a depurar el código.
import traceback

# Para funciones relacionadas con el tiempo, como medir cuánto tarda en ejecutarse una simulación.
import time

# ==============================================================================
# INICIO: Contenido de Celda 0 (Setup Global)
# Esta sección prepara el entorno y define elementos globales.
# ==============================================================================

# --- Opciones de Visualización y Estilos Gráficos ---

# Configuración para pandas:
# 'display.max_rows', None: Muestra todas las filas de un DataFrame.
# 'display.max_columns', None: Muestra todas las columnas de un DataFrame.
# 'display.max_colwidth', None: Muestra el contenido completo de las celdas sin truncarlo.
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# Intenta usar un estilo específico de Matplotlib para los gráficos ('seaborn-v0_8-whitegrid').
# Este estilo afecta la apariencia de los gráficos (colores, grillas, etc.).
try:
    plt.style.use('seaborn-v0_8-whitegrid')
# Si el estilo preferido no se encuentra (puede pasar en diferentes versiones de Matplotlib),
# usa un estilo alternativo ('ggplot') y muestra una advertencia.
except OSError:
    print("Advertencia: Estilo 'seaborn-v0_8-whitegrid' no encontrado, usando 'ggplot'.")
    plt.style.use('ggplot')

# Fin del Bloque 1

--- Ejecutando Código Combinado (Setup Global + Simulación Interactiva) ---
--- VERSIÓN: ode_B_modK para Biodiversidad | C_max_stress Base = 150.0 ---


C:\Users\home\anaconda3\envs\condaXBS\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# --- Diccionarios Globales ---
# Mensaje para indicar la inicialización de variables importantes.
print("Inicializando diccionarios globales 'all_widgets' y 'results'...")

# 'all_widgets': Este diccionario almacenará todos los widgets interactivos
# (como sliders, botones, campos de texto) que se crearán para la interfaz gráfica.
# Guardar una referencia a ellos aquí permite acceder a sus valores o modificarlos
# desde diferentes partes del código.
all_widgets = {}

# 'results': Este diccionario almacenará los resultados de la simulación.
# Cada clave corresponde a una variable del modelo (ej. 'D' para Disponibilidad de Agua)
# o a información relevante de la simulación.
# Inicialmente se llenan con 'None' y se actualizarán después de ejecutar la simulación.
results = {
    't': None,       # Array de tiempo de la simulación
    'D': None,       # Array de resultados para Disponibilidad de Agua
    'V': None,       # Array de resultados para Volumen de Salmuera
    'C': None,       # Array de resultados para Calidad del Agua
    'B': None,       # Array de resultados para Biodiversidad
    'S': None,       # Array de resultados para Percepción Social
    'L': None,       # Array de resultados para Concentración de Litio
    'E': None,       # Array de resultados para Beneficios Económicos
    'IP': None,      # Array de resultados para Impacto Potencial
    'SH': None,      # Array de resultados para Seguridad Hídrica
    'FD': None,      # Array de resultados para Factor de Dilución
    'sim_time_used': None, # Tiempo de simulación configurado
    'effective_D_V_inicial': None, # Valor inicial de D(0) usado en la simulación
    # Nombres de los modelos/versiones usados para cada variable (metadatos)
    'model_D': None, 'model_C': None, 'model_B': None, 'model_S': None,
    'model_V': None, 'model_L': None, 'model_E': None, 'model_IP': None,
    'model_SH': None, 'model_FD': None
}

# --- Función para formatear valores numéricos ---
def format_param(value, decimals=2, sci_limit=1e5):
    """
    Formatea un valor numérico para una visualización clara y concisa.

    Args:
        value (float or int): El valor numérico a formatear.
        decimals (int, optional): Número de decimales para números de punto flotante.
                                  Por defecto es 2.
        sci_limit (float, optional): Límite (en valor absoluto) a partir del cual
                                     se usa notación científica. Por defecto es 1e5.

    Returns:
        str: El valor formateado como una cadena de texto.
             Devuelve "N/A" si el valor es None o no finito.
             Devuelve "ErrorFormat" si ocurre un error de tipo.
    """
    # Manejo de casos donde el valor no es válido para formatear.
    if value is None or not math.isfinite(value): # math.isfinite comprueba si no es NaN o infinito
        return "N/A"
    try:
        # Intentar convertir el valor a tipo float para el formateo.
        value = float(value)
        # Condición para usar notación científica:
        # 1. Si el valor absoluto es mayor que sci_limit (números muy grandes).
        # 2. O si el valor absoluto es muy pequeño (menor que 10^-decimals) pero no es cero.
        if abs(value) > sci_limit or (abs(value) < 10**(-decimals) and value != 0):
            return f"{value:.{decimals}e}" # Formato científico, ej: "1.23e+08"
        # Si el valor es un entero (no tiene parte decimal significativa).
        if value == int(value):
            return f"{int(value)}" # Formato entero, ej: "123"
        # Para otros números de punto flotante, usar el número de decimales especificado.
        return f"{value:.{decimals}f}" # Formato flotante, ej: "123.45"
    except (ValueError, TypeError):
        # Si ocurre un error durante la conversión o formateo.
        return "ErrorFormat"

# Fin del Bloque 2

Inicializando diccionarios globales 'all_widgets' y 'results'...


In [3]:
# --- Función Auxiliar para Crear Widgets de Porcentaje/Multiplicador ---
def create_percentage_widget(name, base_value, description, unit="", help_text="", min_mult=0.0, max_mult=2.0, step_mult=0.01):
    """
    Crea un conjunto de widgets (HBox) para un parámetro que se ajusta
    mediante un valor base y un multiplicador.

    Incluye:
    - Etiqueta con la descripción del parámetro.
    - Campo de texto (FloatText) para el valor base.
    - Slider (FloatSlider) para el multiplicador.
    - Etiqueta para mostrar el valor efectivo calculado (base * multiplicador).
    - Texto de ayuda (HTML).

    Args:
        name (str): Nombre clave del parámetro (usado para all_widgets).
        base_value (float): Valor base por defecto del parámetro.
        description (str): Descripción del parámetro para mostrar al usuario.
        unit (str, optional): Unidad del parámetro (ej. "m³", "mg/L"). Por defecto "".
        help_text (str, optional): Texto de ayuda detallado (tooltip). Por defecto "".
        min_mult (float, optional): Mínimo valor del multiplicador. Por defecto 0.0.
        max_mult (float, optional): Máximo valor del multiplicador. Por defecto 2.0.
        step_mult (float, optional): Paso del slider del multiplicador. Por defecto 0.01.

    Returns:
        widgets.HBox: Un contenedor horizontal con los widgets configurados.
    """
    # Accede al diccionario global para almacenar referencias a los widgets.
    global all_widgets

    # Estilos y layouts para los componentes del widget.
    # 'description_width': 'initial' permite que la descripción de la etiqueta se ajuste automáticamente.
    style = {'description_width': 'initial'}
    # Layouts para controlar el ancho, margen y alineación de cada elemento.
    layout_desc = widgets.Layout(width='180px', margin='0 2px 0 0', display='flex', justify_content='flex-end', align_items='center')
    layout_text = widgets.Layout(width='100px', margin='0 2px 0 0')
    layout_slider = widgets.Layout(width='160px', margin='0 2px 0 0')
    layout_label = widgets.Layout(width='100px', margin='0 2px 0 0', display='flex', justify_content='flex-start', align_items='center')
    layout_help = widgets.Layout(width='250px', margin='0 0 0 5px', display='flex', align_items='center')

    # Etiqueta descriptiva del parámetro.
    desc_label = widgets.Label(f"{description}:", layout=layout_desc, tooltip=help_text or description)

    # --- Asegurar que el Valor Base sea Finito ---
    # Si base_value no es un número válido (ej. NaN, infinito, o no convertible a float),
    # se usa 0.0 como valor por defecto para evitar errores.
    safe_base_value = 0.0
    try:
        float_val = float(base_value)
        if math.isfinite(float_val): # Comprueba que sea un número finito.
            safe_base_value = float_val
    except (ValueError, TypeError):
        safe_base_value = 0.0 # Valor por defecto en caso de error de conversión.
    # --- Fin Asegurar Valor Base ---

    # --- Calcular un Paso Adecuado para el FloatText del Valor Base ---
    # Esto permite que el ajuste con las flechas del FloatText sea más útil
    # según la magnitud del número base.
    if abs(safe_base_value) > 1e-6: # Si el valor base no es demasiado cercano a cero.
        # El paso es el 1% del valor base, con límites superior e inferior.
        text_step_val = max(min(abs(safe_base_value) * 0.01, 1e6), 1e-6)
    elif safe_base_value == 0:
        text_step_val = 0.01 # Un paso pequeño si el valor base es cero.
    else: # Para valores muy pequeños pero no cero.
        text_step_val = 1e-6
    # --- Fin Step FloatText ---

    # Campo de texto para ingresar/mostrar el valor base.
    base_text = widgets.FloatText(value=safe_base_value, description="", layout=layout_text, style=style, step=text_step_val)
    # Tooltip simple para el slider.
    simple_tooltip = help_text or f"Ajustar multiplicador ({min_mult} a {max_mult})"

    # --- Asegurar Parámetros Finitos y Válidos para FloatSlider ---
    # Convierte los límites y el paso del slider a float y maneja valores no finitos.
    safe_min_mult = float(min_mult) if math.isfinite(min_mult) else 0.0
    safe_max_mult = float(max_mult) if math.isfinite(max_mult) else 2.0
    safe_step_mult = float(step_mult) if math.isfinite(step_mult) and step_mult > 0 else 0.01

    # Asegura que el rango del slider sea válido (max > min).
    slider_range = safe_max_mult - safe_min_mult
    if slider_range <= 0: # Si max no es mayor que min, ajusta max.
        safe_max_mult = safe_min_mult + 1.0
        slider_range = 1.0
    # Asegura que el paso del slider sea válido y razonable para el rango.
    if safe_step_mult <= 0 or safe_step_mult > slider_range:
        safe_step_mult = slider_range / 100.0 # Un paso por defecto del 1% del rango.
    safe_step_mult = max(safe_step_mult, 1e-9) # Paso mínimo para evitar problemas.

    # Valor inicial para el slider del multiplicador (usualmente 1.0 para 100%).
    initial_slider_value = 1.0
    # Si 1.0 está fuera del rango [min, max] del slider, ajusta al mínimo.
    if not (safe_min_mult <= initial_slider_value <= safe_max_mult):
        initial_slider_value = safe_min_mult
    # --- Fin Asegurar Parámetros Slider ---

    # Slider para el multiplicador.
    multiplier_slider = widgets.FloatSlider(
        value=initial_slider_value, # Valor inicial (ej. 1.0)
        min=safe_min_mult,          # Mínimo del slider
        max=safe_max_mult,          # Máximo del slider
        step=safe_step_mult,        # Incremento del slider
        description="",             # Sin descripción propia, ya la tiene desc_label
        layout=layout_slider,
        readout=False,              # No muestra el valor numérico junto al slider (se usa effective_value_label)
        style=style,
        tooltip=simple_tooltip      # Texto de ayuda al pasar el mouse
    )

    # Calcula el valor efectivo inicial (base * multiplicador).
    initial_effective_value = safe_base_value * initial_slider_value
    # Etiqueta para mostrar el valor efectivo, formateado.
    effective_value_label = widgets.Label(f"{format_param(initial_effective_value)} {unit}", layout=layout_label)

    # Widget HTML para mostrar el texto de ayuda de forma más elaborada si es necesario.
    help_widget = widgets.HTML(value=f'<small style="color:#555;"><i>{help_text}</i></small>', layout=layout_help)

    # Almacena referencias a los widgets de valor base y multiplicador en el diccionario global.
    # Esto permite acceder a sus valores desde otras partes del código.
    all_widgets[name + "_base"] = base_text
    all_widgets[name + "_multiplier"] = multiplier_slider

    # --- Función para actualizar el valor efectivo cuando cambian base o multiplicador ---
    def update_effective_value(*args):
        """Actualiza la etiqueta del valor efectivo."""
        base = base_text.value
        multiplier = multiplier_slider.value
        # Verifica si los valores son números válidos.
        if base is None or not math.isfinite(base) or multiplier is None or not math.isfinite(multiplier):
            effective_value_label.value = "Inválido" # Muestra "Inválido" si hay error.
            return
        # Calcula y actualiza la etiqueta del valor efectivo.
        effective_value = base * multiplier
        effective_value_label.value = f"{format_param(effective_value)} {unit}"

    # "Observa" los cambios en los widgets 'base_text' y 'multiplier_slider'.
    # Cuando el atributo 'value' de alguno de ellos cambia, se llama a 'update_effective_value'.
    base_text.observe(update_effective_value, names='value')
    multiplier_slider.observe(update_effective_value, names='value')

    # Devuelve un HBox (contenedor horizontal) con todos los elementos creados.
    return widgets.HBox([desc_label, base_text, multiplier_slider, effective_value_label, help_widget])

# Fin del Bloque 3

In [4]:
# ==============================================================================
# Bloque 4: Funciones Auxiliares Específicas para los Flujos y Cálculos del Modelo (VERSIÓN REVISADA)
# ==============================================================================

# (Asegúrate de que las importaciones como numpy y math estén hechas en Bloque 1)

def convert_precip_evap_vol(rate_mm_month, area_km2):
    """
    Convierte una tasa dada en mm/mes y un área en km² a un volumen en m³/año.
    (Sin cambios respecto a la versión anterior)
    """
    if area_km2 is None or not math.isfinite(area_km2) or \
       rate_mm_month is None or not math.isfinite(rate_mm_month):
        return 0.0
    try:
        rate_mm_year = float(rate_mm_month) * 12.0
        rate_m_year = rate_mm_year * 0.001
        area_m2 = float(area_km2) * 1e6
        return rate_m_year * area_m2
    except:
        return 0.0

# --- DATOS MENSUALES PROMEDIO (Sin cambios) ---
avg_monthly_precip_mm = np.array([
    83.1, 73.2, 29.0,  2.0,  1.6,  0.3,
     1.9,  1.0,  0.4,  3.0,  2.2, 31.6
])
avg_monthly_evap_mm = np.array([
    142.9, 124.5, 129.6, 104.0,  79.4,  62.9,
     67.4,  85.4, 108.4, 138.4, 151.6, 155.7
])

def get_precipitacion_mensual_vol(t, params):
    """
    Calcula el volumen de precipitación anualizado para un tiempo t.
    (Sin cambios respecto a la versión anterior)
    """
    mes_idx = int(np.floor(t * 12) % 12)
    tasa_mm_mes = avg_monthly_precip_mm[mes_idx]
    area = params.get('D_area', 10582) # Nombre actualizado del parámetro de área para D(t)
    multiplier = params.get("D_Pcp_multiplier", 1.0)
    return convert_precip_evap_vol(tasa_mm_mes * multiplier, area)

def get_evaporacion_mensual_vol(t, params):
    """
    Calcula el volumen de evaporación anualizado para un tiempo t.
    (Sin cambios respecto a la versión anterior)
    """
    mes_idx = int(np.floor(t * 12) % 12)
    tasa_mm_mes = avg_monthly_evap_mm[mes_idx]
    area = params.get('D_area', 10582) # Nombre actualizado del parámetro de área para D(t)
    multiplier = params.get("D_Evap_multiplier", 1.0)
    return convert_precip_evap_vol(tasa_mm_mes * multiplier, area)

def evaporacion_no_lineal_factor(D, params, v_init_key="D_V_inicial"):
    """
    Calcula un factor (0-1) que modula la evaporación de agua dulce.
    (Sin cambios respecto a la versión anterior, pero 'D_V_inicial' es el nombre del parámetro en df_params)
    """
    D_safe = max(D, 0.0)
    ref_val = max(params.get("D_ref_evap", params.get(v_init_key, 1e-6)), 1e-6)
    factor = np.clip(D_safe / ref_val, 0.0, 1.0)
    return factor

def extraccion_logistica(D, params, extrac_key, v_init_key="D_V_inicial"):
    """
    Modela una tasa de extracción que disminuye logísticamente si D(t) es bajo.
    (Sin cambios respecto a la versión anterior, pero 'D_V_inicial' es el nombre del parámetro en df_params)
    """
    D_safe = max(D, 0.0); E_max = params.get(extrac_key, 0.0)
    if E_max is None or not math.isfinite(E_max): E_max = 0.0
    v_init_val = params.get(v_init_key, 1e-6)
    if v_init_val is None or not math.isfinite(v_init_val): v_init_val = 1e-6
    crit_key_specific = f"{extrac_key}_critico"; crit_key_general = "D_parametro_critico"
    default_crit_val = v_init_val * 0.2
    parametro_critico = params.get(crit_key_specific, params.get(crit_key_general, default_crit_val))
    k = 0.01; factor = 0.0
    try:
        exp_arg = -k * (D_safe - parametro_critico)
        denominator = 1.0 + np.exp(np.clip(exp_arg, -700, 700))
        factor = np.where(abs(denominator) > 1e-9, 1.0 / denominator, 1.0 if D_safe > parametro_critico else 0.0)
    except FloatingPointError: factor = 1.0 if D_safe > parametro_critico else 0.0
    return E_max * factor

# --- Funciones para Flujos Temporales Cíclicos (Sinusoidales) ---
def R_ns_t(t, params): # Recarga Natural de Salmuera
    R_base=params.get("R_ns_base",0); A_R=params.get("A_R",0)
    T_R=params.get("T_R_V",1); phi_R=params.get("phi_R_V",0)
    T_R_safe=max(T_R, 1e-6); return R_base * (1 + A_R * np.sin((2*np.pi*t/T_R_safe)+phi_R))

# ##### INICIO DE MODIFICACIÓN IMPORTANTE para E_EDL_s_t #####
def E_EDL_s_t(t, params): # Extracción de Salmuera para EDL (AHORA CALCULADA)
    """
    Calcula la tasa instantánea de extracción de salmuera (m³/año).
    La tasa BASE de extracción se calcula a partir de:
    - E_Q_manual (producción LCE deseada)
    - L_Planning_Ref_Concentration_base (concentración Li de planificación)
    - L_DLE_RecoveryEff_base (eficiencia de recuperación EDL)
    A esta base calculada se le aplica una variación sinusoidal.
    """
    # 1. Obtener los parámetros para el cálculo del caudal base de salmuera
    target_lce_production_annual = params.get("E_Q_manual", 0.0) # ton LCE/año
    
    planning_li_conc_mg_L = params.get("L_Planning_Ref_Concentration_base", 600.0) # mg/L (default si no está)
    planning_li_conc_g_m3 = planning_li_conc_mg_L # 1 mg/L = 1 g/m³
    
    dle_recovery_eff = params.get("L_DLE_RecoveryEff_base", 0.85) # fracción (default si no está)

    # 2. Calcular la cantidad de Litio metal necesario (en gramos por año)
    LCE_to_Li_metal_ton_factor = 1.0 / 5.323 # ton Li-metal / ton LCE
    g_per_ton_metric = 1000000.0 # gramos por tonelada métrica
    
    target_li_metal_g_year = target_lce_production_annual * LCE_to_Li_metal_ton_factor * g_per_ton_metric
    
    # 3. Calcular la tasa base de extracción de salmuera (m³/año)
    calculated_V_E_EDL_s_base = 0.0
    # Solo extraer si hay producción deseada, concentración de planificación > 0 y eficiencia > 0
    if target_lce_production_annual > 1e-6 and \
       planning_li_conc_g_m3 > 1e-6 and \
       dle_recovery_eff > 1e-6:
        calculated_V_E_EDL_s_base = target_li_metal_g_year / (planning_li_conc_g_m3 * dle_recovery_eff)
    
    # 4. Aplicar la variación sinusoidal a esta base calculada (usando parámetros existentes)
    A_E = params.get("V_A_E", 0.3)      # Amplitud de la variación
    T_E = params.get("T_E_V", 1.0)      # Periodo de la variación
    phi_E = params.get("phi_E_V", 0.5)  # Fase de la variación
    T_E_safe = max(T_E, 1e-6)           # Evitar división por cero
    
    current_brine_extraction_rate = calculated_V_E_EDL_s_base * \
                                    (1 + A_E * np.sin((2 * np.pi * t / T_E_safe) + phi_E))
                                    
    return current_brine_extraction_rate
# ##### FIN DE MODIFICACIÓN IMPORTANTE para E_EDL_s_t #####

def Ev_s_t(t, params): # Evaporación de Salmuera
    Ev_base=params.get("Ev_s_base",0); A_Ev=params.get("A_Ev",0)
    T_Ev=params.get("T_Ev_V",1); phi_Ev=params.get("phi_Ev_V",0)
    T_Ev_safe=max(T_Ev, 1e-6); return Ev_base * (1 + A_Ev * np.sin((2*np.pi*t/T_Ev_safe)+phi_Ev))

def P_t(t, params): # Precio del Litio
    P_base=params.get("E_P_base",0); A=params.get("E_A",0)
    T=params.get("E_T",1); phi=params.get("E_phi",0)
    T_safe=max(T,1e-6); return P_base*(1+A*np.sin((2*np.pi*t/T_safe)+phi))

# --- Funciones para Cálculos Económicos y de Impacto ---
def C_u(t, params, t_interp, L_arr): # Costo Operativo Unitario
    L_val = np.interp(t, t_interp, L_arr); L_val = max(L_val, 1e-6)
    C_base=params.get("E_C_base",0); L_ref=max(params.get("E_L_ref",1),1e-6)
    alpha=params.get("E_alpha",1); ratio = L_ref / L_val
    cost_factor = np.power(ratio, alpha) if ratio < 1e6 else 1e18; return C_base * cost_factor

def Q_dinamica(t, params, t_interp, V_arr, L_arr): # Producción Anual Dinámica
    Q_max = params.get("E_Q_manual", 0); V_val = np.interp(t, t_interp, V_arr); L_val = np.interp(t, t_interp, L_arr)
    V_thresh = max(params.get("V_thresh_Q_base", 1e-6), 1e-6); L_thresh = max(params.get("L_thresh_Q_base", 1e-6), 1e-6)
    factor_V = np.clip(V_val / V_thresh, 0.0, 1.0); factor_L = np.clip(L_val / L_thresh, 0.0, 1.0)
    Q_eff = Q_max * factor_V * factor_L; return Q_eff

def Costos_Externalidades(t, params, t_interp, C_arr, B_arr, S_arr):
    C_val = np.interp(t, t_interp, C_arr); B_val = np.interp(t, t_interp, B_arr); S_val = np.interp(t, t_interp, S_arr)
    c1 = params.get("E_c1_base", 0); c2 = params.get("E_c2_base", 0); c3 = params.get("E_c3_base", 0) # Usa los _base para los coeficientes
    C_crit_E = params.get("C_crit_E_base", 1e6); B_max_E = params.get("B_max_E_base", 0); S_max_E = params.get("S_max_E_base", 0) # Usa los _base para los umbrales
    costo_C = c1 * max(0, C_val - C_crit_E); costo_B = c2 * max(0, B_max_E - B_val); costo_S = c3 * max(0, S_max_E - S_val)
    costo_total_ext = costo_C + costo_B + costo_S; return costo_total_ext

# ##### INICIO DE MODIFICACIÓN IMPORTANTE para Utilidad_Neta #####
def Utilidad_Neta(t, params, t_interp, L_arr, V_arr, C_arr, B_arr, S_arr):
    """Calcula la utilidad neta instantánea, incluyendo costos fijos operativos."""
    price = P_t(t, params)
    cost_op_unit = C_u(t, params, t_interp, L_arr) # Costo operativo variable unitario
    produccion = Q_dinamica(t, params, t_interp, V_arr, L_arr) # Producción real
    
    costos_ext = Costos_Externalidades(t, params, t_interp, C_arr, B_arr, S_arr) # Costos por externalidades
    
    # NUEVO: Obtener y restar el Costo Fijo Operativo Anual
    costo_fijo_anual = params.get("E_CostoFijoOperativoAnual_base", 0.0) # u.m./año
    
    # Utilidad = (Ingresos por ventas) - (Costos Variables de Producción) - (Costos de Externalidades) - (Costos Fijos Anuales)
    # Nota: (price - cost_op_unit) es el margen bruto unitario.
    utilidad = (price * produccion) - (cost_op_unit * produccion) - costos_ext - costo_fijo_anual
    # Una forma alternativa de escribirlo, si C_u ya es el costo variable total por unidad producida:
    # utilidad = (price - cost_op_unit) * produccion - costos_ext - costo_fijo_anual
    # La forma (price * Q) - (cost_op_unit * Q) es más clara para separar ingresos de costos variables.
    
    return utilidad
# ##### FIN DE MODIFICACIÓN IMPORTANTE para Utilidad_Neta #####

# --- Funciones para calcular Desviaciones (usadas en IP) --- (Sin cambios)
def dev_D(t, params): D_val=np.interp(t,results['t'],results['D']); D_ref=max(params.get("IP_D_ref",1e6),1e-6); return np.maximum(0,(D_ref-D_val)/D_ref)
def dev_C(t, params): C_val=np.interp(t,results['t'],results['C']); C_ref=max(params.get("IP_C_ref",1e6),1e-6); return np.maximum(0,(C_val/C_ref)-1)
def dev_B(t, params): B_val=np.interp(t,results['t'],results['B']); B_max=max(params.get("IP_B_max",1e6),1e-6); return np.maximum(0,(B_max-B_val)/B_max)
def dev_S(t, params): S_val=np.interp(t,results['t'],results['S']); S_max=max(params.get("IP_S_max",1e6),1e-6); return np.maximum(0,(S_max-S_val)/S_max)
def dev_E(t, params): E_val=np.interp(t,results['t'],results['E']); E_crit=max(params.get("IP_E_crit",0),1e-6); return np.maximum(0,(E_crit-E_val)/E_crit if abs(E_crit) > 1e-9 else (-E_val / 1e-9 if E_val < 0 else 0.0) ) # Manejo más robusto de E_crit=0
def dev_V(t, params): V_val=np.interp(t,results['t'],results['V']); V_ref=max(params.get("IP_V_ref",1e6),1e-6); return np.maximum(0,(V_ref-V_val)/V_ref)
def dev_L(t, params): L_val=np.interp(t,results['t'],results['L']); L_ref=max(params.get("IP_L_ref_ip",1e6),1e-6); return np.maximum(0,(L_ref-L_val)/L_ref)

# Fin del Bloque 4 (REVISADO)

In [5]:
# ==============================================================================
# Bloque 5: Definición de las Ecuaciones Diferenciales Ordinarias (EDOs) (VERSIÓN REVISADA)
# ==============================================================================

# (Asegúrate de que las funciones auxiliares del Bloque 4 estén definidas
# y que las importaciones necesarias como numpy y math estén hechas)

def ode_D(t, D_in, params):
    """
    Calcula la tasa de cambio de la Disponibilidad de Agua dulce (D).
    MODIFICADO: extr_EDL_agua ahora se calcula usando E_Q_manual y D_AguaPorTonLCE_base.
    """
    D_safe = max(D_in[0], 0.0)
    precip = get_precipitacion_mensual_vol(t, params)
    Af_base = params.get("D_Af_base", 0.0)
    Rn_base = params.get("D_Rn_base", 0.0)
    evap_base_mensual = get_evaporacion_mensual_vol(t, params)
    factor_nl_evap = evaporacion_no_lineal_factor(D_safe, params, v_init_key="D_V_inicial") # D_V_inicial es el nombre en df_params
    evap = evap_base_mensual * factor_nl_evap

    # --- INICIO DE MODIFICACIÓN para extr_EDL_agua ---
    q_manual_production = params.get("E_Q_manual", 0.0)       # ton LCE/año
    agua_por_ton_lce = params.get("D_AguaPorTonLCE_base", 0.0) # m³/ton LCE (nuevo parámetro de Bloque 6)
    extr_EDL_agua = q_manual_production * agua_por_ton_lce     # m³/año
    # --- FIN DE MODIFICACIÓN para extr_EDL_agua ---
        
    extr_Otros = extraccion_logistica(D_safe, params, extrac_key="D_Extr_Otros_Usos", v_init_key="D_V_inicial")
    
    deriv = (precip + Af_base + Rn_base) - (evap + extr_EDL_agua + extr_Otros)

    if D_safe <= 1e-9 and deriv < 0: return [0.0]
    return [deriv]

def ode_V(t, V_in, params):
    """
    Calcula la tasa de cambio del Volumen de Salmuera (V).
    (Sin cambios estructurales aquí, pero E_EDL_s_t del Bloque 4 ahora es dinámico)
    """
    V_safe = max(V_in[0], 0.0); R = R_ns_t(t, params); E = E_EDL_s_t(t, params); Ev = Ev_s_t(t, params)
    V_scale = max(params.get("V_scale", 1e-6), 1e-6);
    extraction_evap = (E + Ev) * (V_safe / (V_safe + V_scale)) if (V_safe + V_scale) > 1e-9 else (E + Ev)
    deriv = R - extraction_evap
    if V_safe <= 1e-9 and deriv < 0: return [0.0]
    return [deriv]

def ode_C(t, C_in, params, t_D, D_arr):
    """
    Calcula la tasa de cambio de la Calidad del Agua (C, indicador de mala calidad).
    MODIFICADO: Las cargas M y X ahora tienen componentes de fondo y por actividad EDL.
    """
    C_safe = max(C_in[0], 0.0)
    D_t = np.interp(t, t_D, D_arr); D_t = max(D_t, 1e-6)
    D_ref = max(params.get("D_ref_C", params.get("D_V_inicial", 1e6)), 1e-6)
    R_dep_yr = params.get("R_dep", 0.0)

    # --- INICIO DE MODIFICACIÓN para Cargas Contaminantes ---
    q_manual_production = params.get("E_Q_manual", 0.0) # ton LCE/año

    # Cargas de fondo (usando los parámetros renombrados de Bloque 6)
    m_background = params.get("M_CargaFondo_base", 0.0) 
    x_background = params.get("X_CargaFondo_base", 0.0) 
    
    # Factores de carga por producción (nuevos parámetros de Bloque 6)
    m_factor_prod = params.get("M_FactorPorProduccion_base", 0.0) # (mg/L/año)/(1000 ton LCE/año)
    x_factor_prod = params.get("X_FactorPorProduccion_base", 0.0) # (mg/L/año)/(1000 ton LCE/año)
    
    reference_production_unit_for_factors = 1000.0 # Asegúrate que esta unidad coincida con la unidad de los factores

    # Carga adicional debido a la actividad EDL
    m_load_from_activity = m_factor_prod * (q_manual_production / reference_production_unit_for_factors)
    x_load_from_activity = x_factor_prod * (q_manual_production / reference_production_unit_for_factors)
    
    total_m_input_rate = m_background + m_load_from_activity
    total_x_input_rate = x_background + x_load_from_activity
        
    # Carga ponderada antes de dilución (parámetros 'a' y 'b' de Bloque 6)
    carga_total_ponderada = (params.get("a",0.5) * total_m_input_rate + \
                             params.get("b",0.3) * total_x_input_rate)
    # --- FIN DE MODIFICACIÓN para Cargas Contaminantes ---
        
    termino_carga_ajustada = carga_total_ponderada * (D_ref / D_t) if D_t > 1e-9 else carga_total_ponderada * 1e9
    deriv = termino_carga_ajustada - R_dep_yr

    if C_safe <= 1e-9 and deriv < 0: return [0.0]
    return [deriv]

def ode_B_modK(t, B_in, params, t_interp, D_arr, C_arr):
    """
    ODE para Biodiversidad B(t) modificando K efectivo.
    (Sin cambios estructurales aquí, pero los valores de D_crit, C_crit, C_max_stress vienen de params actualizados)
    """
    B_safe = max(B_in[0], 0.0)
    D_val = np.interp(t, t_interp, D_arr); C_val = np.interp(t, t_interp, C_arr)
    r = params.get("B_r_base", 0.1); K_base = max(params.get("B_K_base", 1), 1.0)
    D_crit = params.get("D_crit", params.get("D_V_inicial", 1e9) * 0.5) # D_crit es un param simple ahora en df_params
    C_crit = params.get("C_crit", 4.0) # C_crit es un param simple ahora en df_params
    C_max_stress = params.get("C_max_stress", C_crit * 5.0)
    C_max_stress = max(C_max_stress, C_crit + 1e-6)
    factor_D = np.clip(D_val / max(D_crit, 1e-9), 0.0, 1.0)
    factor_C = np.clip(1.0 - max(0, C_val - C_crit) / max(1e-9, C_max_stress - C_crit), 0.0, 1.0)
    K_eff = K_base * factor_D * factor_C; K_eff = max(K_eff, 1e-6)
    crecimiento = r * B_safe * (1 - B_safe / K_eff); dBdt = crecimiento
    if B_safe <= 1e-9 and dBdt < 0: return [0.0]
    return [dBdt]

# ##### INICIO DE MODIFICACIÓN IMPORTANTE para ode_S_refinado #####
def ode_S_refinado(t, S_in, params, t_interp, C_arr, B_arr, D_arr_para_S): # Añadido D_arr_para_S
    """
    Calcula la tasa de cambio de la Percepción Social (S, índice).
    MODIFICADO: Ahora incluye un término de penalización por baja disponibilidad de D(t).
    """
    S_safe = S_in[0]
    C_val = np.interp(t, t_interp, C_arr)
    B_val = np.interp(t, t_interp, B_arr)
    D_val_para_S = np.interp(t, t_interp, D_arr_para_S) # Interpolar D(t) para S

    p1 = params.get("S_p1", 0.05)
    p2 = params.get("S_p2", 0.03)
    p4 = params.get("S_p4", 0.1)
    S_C_crit = params.get("S_C_crit_base", 5.0)

    # Nuevos parámetros y lógica para el impacto de D(t) en S(t)
    # S_D_crit_social_base es el nombre del NUEVO PARAMETRO en df_params (Bloque 6)
    # D_V_inicial se usa para calcular el valor base de S_D_crit_social_base si se definió como 'D(0)*0.3'
    # Aquí, asumimos que S_D_crit_social_base ya tiene su valor numérico en params
    s_d_crit_social_val = params.get("S_D_crit_social_base", params.get("D_V_inicial", 1e11) * 0.3)
    s_p_disp_agua = params.get("S_p_disp_agua_base", 0.05) # NUEVO PARAMETRO de Bloque 6

    influencia_B = p2 * B_val
    penalizacion_C = p1 * max(0, C_val - S_C_crit)
    
    # Nuevo término de penalización por baja disponibilidad de D(t)
    norm_factor_D_social = max(s_d_crit_social_val, 1e-6) # Evitar división por cero
    # Desviación normalizada: 0 si D >= umbral, aumenta hasta 1 si D tiende a 0.
    desviacion_D_social = max(0, (s_d_crit_social_val - D_val_para_S) / norm_factor_D_social)
    penalizacion_D = s_p_disp_agua * desviacion_D_social
    
    decaimiento_S = p4 * S_safe

    deriv = influencia_B - penalizacion_C - penalizacion_D - decaimiento_S # Se añade - penalizacion_D
    return [deriv]
# ##### FIN DE MODIFICACIÓN IMPORTANTE para ode_S_refinado #####

def ode_L(t, L_in, params, t_interp, V_arr, V_deriv_arr):
    """
    Calcula la tasa de cambio de la Concentración de Litio (L) en salmuera.
    (Sin cambios estructurales aquí, pero E_EDL_s_t y dVdt_t vienen de funciones/ODEs actualizadas)
    """
    L_safe = max(L_in[0], 0.0); V_t = np.interp(t, t_interp, V_arr); V_t = max(V_t, 1e-6); dVdt_t = np.interp(t, t_interp, V_deriv_arr)
    E_s_t = E_EDL_s_t(t, params) # Esta función E_EDL_s_t fue modificada en Bloque 4
    aditivos = params.get("L_Aditivos", 0); gamma = params.get("L_gamma", 0); L_eq = params.get("L_eq", 0)
    aditivos_masa_rate = aditivos * V_t
    dLdt = (aditivos_masa_rate - L_safe*E_s_t - gamma*(L_safe-L_eq)*V_t - L_safe*dVdt_t) / V_t if V_t > 1e-9 else 0
    if L_safe <= 1e-9 and dLdt < 0: return [0.0]
    return [dLdt]

def ode_E(t, E_in, params, t_interp, L_arr, V_arr, C_arr, B_arr, S_arr):
    """
    Calcula la tasa de cambio del Beneficio Económico acumulado (E).
    (Sin cambios estructurales aquí, pero Utilidad_Neta del Bloque 4 ahora incluye costos fijos)
    """
    # E_in[0] es el beneficio acumulado, su derivada es la utilidad neta instantánea.
    # La función Utilidad_Neta (Bloque 4) ya fue modificada para incluir E_CostoFijoOperativoAnual_base.
    # La condición inicial E0 = -CAPEX se manejará en _core_simulation_full (Bloque 8).
    dEdt = Utilidad_Neta(t, params, t_interp, L_arr, V_arr, C_arr, B_arr, S_arr)
    return [dEdt]

def ode_IP_full(t, IP_in, params):
    """
    Calcula la tasa de cambio del Impacto Potencial agregado (IP, índice).
    (Sin cambios estructurales aquí. Su comportamiento cambiará por los cambios en D, C, B, S, E, V, L)
    """
    IP_safe = max(IP_in[0], 0.0)
    term = (params.get("IP_ip_alpha",0) * dev_D(t, params) +
            params.get("IP_ip_beta",0)  * dev_C(t, params) +
            params.get("IP_ip_gamma",0) * dev_B(t, params) +
            params.get("IP_ip_delta",0) * dev_S(t, params) +
            params.get("IP_ip_epsilon",0)* dev_E(t, params) +
            params.get("IP_ip_zeta",0)  * dev_V(t, params) +
            params.get("IP_ip_eta",0)   * dev_L(t, params))
    lambda_IP = params.get("IP_lambda_IP",0)
    deriv = term - lambda_IP * IP_safe
    if IP_safe <= 1e-9 and deriv < 0: return [0.0]
    return [deriv]

def ode_SH_CalidadInversa(t, SH_in, params, t_interp, D_arr, C_arr):
    """
    Calcula la tasa de cambio de la Seguridad Hídrica (SH, índice).
    (Sin cambios estructurales aquí. Su comportamiento cambiará por los cambios en D y C.
     Se debe revisar y justificar SH_psi, SH_D_ref, SH_C_ref en la tesis).
    """
    SH_safe = max(SH_in[0], 0.0); D_val = np.interp(t, t_interp, D_arr); C_val = np.interp(t, t_interp, C_arr)
    D_ref = max(params.get("SH_D_ref", 1e6), 1e-6); C_ref = max(params.get("SH_C_ref", 1e-6), 1e-6)
    phi = params.get("SH_phi", 0); psi = params.get("SH_psi", 0)
    quality_factor = np.maximum(0.0, 1.0 - (C_val / C_ref))
    availability_factor = D_val / D_ref if D_ref > 1e-9 else 0
    pos_term = phi * availability_factor * quality_factor
    deriv = pos_term - psi * SH_safe
    if SH_safe <= 1e-9 and deriv < 0: return [0.0]
    return [deriv]

def ode_FD(t, FD_in, params, t_interp, D_arr):
    """
    Calcula la tasa de cambio del Factor de Dilución (FD, adimensional).
    (Sin cambios estructurales aquí. Su rol como indicador de salida suavizado debe aclararse en la tesis).
    """
    FD_safe = max(FD_in[0], 0.0); D_val = np.interp(t, t_interp, D_arr)
    D_ref = max(params.get("FD_D_ref", 1e-6), 1e-6); kappa = params.get("FD_kappa", 0)
    target = D_val / D_ref if D_ref > 1e-9 else 0
    deriv = kappa * (target - FD_safe)
    return [deriv]

print("Bloque 5: Definiciones de funciones ODE REVISADAS.") # Mensaje actualizado

# Fin del Bloque 5 (REVISADO)

Bloque 5: Definiciones de funciones ODE REVISADAS.


In [6]:
# ==============================================================================
# Bloque 6: Definición y Carga de Parámetros del Modelo (VERSIÓN REVISADA)
# ==============================================================================

# --- Lista de nombres de parámetros para referencia interna y organización ---
# (Estas listas deben actualizarse para reflejar los cambios)

# Nombres de parámetros que se controlan con un valor base y un multiplicador (tipo '%')
percentage_param_names_full_REVISADO = [
    'D_area', 'D_V_inicial', 'D_Pcp_multiplier', 'D_Evap_multiplier',
    'D_Af_base', 'D_Rn_base',
    'D_AguaPorTonLCE_base', # NUEVO (reemplaza D_Extr_EDL_agua)
    'D_Extr_Otros_Usos',
    'V_inicial_sal', 'R_ns_base', 'A_R', 'T_R_V', 'phi_R_V',
    # V_E_EDL_s_base ELIMINADO como entrada directa
    'V_A_E', 'T_E_V', 'phi_E_V', # Estos ahora aplican a la base calculada de extracción de salmuera
    'L_Planning_Ref_Concentration_base', # NUEVO
    'L_DLE_RecoveryEff_base',          # NUEVO
    'Ev_s_base', 'A_Ev', 'T_Ev_V', 'phi_Ev_V', 'V_scale',
    'C_inicial', 'a', 'b',
    'M_CargaFondo_base',               # NUEVO (reemplaza M)
    'X_CargaFondo_base',               # NUEVO (reemplaza X)
    'M_FactorPorProduccion_base',      # NUEVO
    'X_FactorPorProduccion_base',      # NUEVO
    'R_dep',
    'B_inicial', 'B_r_base', 'B_K_base','C_max_stress',
    'S_inicial', 'S_p1', 'S_p2', 'S_p4', 'S_C_crit_base',
    'S_D_crit_social_base',            # NUEVO
    'S_p_disp_agua_base',              # NUEVO
    'L_Aditivos', 'L_inicial', 'L_gamma', 'L_eq',
    'E_P_base', 'E_A', 'E_T', 'E_phi', 'E_C_base', 'E_L_ref', 'E_alpha', 'E_Q_manual',
    'V_thresh_Q_base', 'L_thresh_Q_base',
    'E_CostoFijoOperativoAnual_base', # NUEVO
    'E_CAPEX_base',                   # NUEVO
    'E_c1_base', 'E_c2_base', 'E_c3_base',
    'C_crit_E_base', 'B_max_E_base', 'S_max_E_base',
    'IP_inicial', 'IP_lambda_IP', 'IP_D_ref', 'IP_C_ref', 'IP_B_max', 'IP_S_max', 'IP_E_crit', 'IP_V_ref', 'IP_L_ref_ip',
    'IP_ip_alpha', 'IP_ip_beta', 'IP_ip_gamma', 'IP_ip_delta', 'IP_ip_epsilon', 'IP_ip_zeta', 'IP_ip_eta',
    'SH_inicial', 'SH_D_ref', 'SH_C_ref', 'SH_phi', 'SH_psi',
    'FD_inicial', 'FD_D_ref', 'FD_kappa'
]
# Nombres de parámetros que se controlan con un valor simple
simple_param_names_full_REVISADO = ['sim_time', 'm', 'n', 'D_crit', 'C_crit'] # m y n siguen como NO USADOS

# --- DataFrame Resumen de Parámetros (VERSIÓN REVISADA) ---
print("\n--- Generando Resumen de Parámetros REVISADO (Post-Barrido Conceptual) ---")
parameter_data_REVISADO = [
    # Global
    {'Bloque': 'Global', 'Parámetro': 'sim_time', 'Descripción': 'Tiempo Simulación', 'Valor Base': 50, 'Unidad': 'años', 'Tipo': 'Simple', 'Ayuda': 'Duración total de la simulación.'},

    # --- D (Disponibilidad de Agua Dulce) ---
    {'Bloque': 'D', 'Parámetro': 'D_area', 'Descripción': 'Área Cuenca (D)', 'Valor Base': 10582, 'Unidad': 'km²', 'Tipo': '%', 'Ayuda': 'Área superficial (Salar) para Pcp/Evap que afectan D(t).'},
    {'Bloque': 'D', 'Parámetro': 'D_V_inicial', 'Descripción': 'Volumen Inicial D(0)', 'Valor Base': 3.75E+11, 'Unidad': 'm³', 'Tipo': '%', 'Ayuda': 'Volumen agua dulce inicial. Ref. para Evap no lineal y umbrales.'},
    {'Bloque': 'D', 'Parámetro': 'D_Pcp_multiplier', 'Descripción': 'Multiplicador Pcp Mensual', 'Valor Base': 1.0, 'Unidad': '', 'Tipo': '%', 'Ayuda': 'Escala datos Pcp mensual. (>1 más húmedo, <1 más seco).', 'min_mult': 0.5, 'max_mult': 1.5, 'step_mult': 0.05},
    {'Bloque': 'D', 'Parámetro': 'D_Af_base', 'Descripción': 'Afluentes Base (Af)', 'Valor Base': 6.601e7, 'Unidad': 'm³/año', 'Tipo': '%', 'Ayuda': 'Caudal base ríos superficiales (Ref: Tesis Tabla 2.1).'},
    {'Bloque': 'D', 'Parámetro': 'D_Rn_base', 'Descripción': 'Recarga Nat. Base (Rn)', 'Valor Base': 6.83e6, 'Unidad': 'm³/año', 'Tipo': '%', 'Ayuda': 'Recarga subterránea base agua dulce (Ref: Tesis Tablas 1.2/2.1). Alta incertidumbre.'},
    {'Bloque': 'D', 'Parámetro': 'D_Evap_multiplier', 'Descripción': 'Multiplicador Evap Mensual', 'Valor Base': 1.0, 'Unidad': '', 'Tipo': '%', 'Ayuda': 'Escala datos Evap mensual. (>1 mayor demanda evaporativa).', 'min_mult': 0.5, 'max_mult': 1.5, 'step_mult': 0.05},
    # NUEVO PARÁMETRO (reemplaza D_Extr_EDL_agua como total anual)
    {'Bloque': 'D', 'Parámetro': 'D_AguaPorTonLCE_base', 'Descripción': 'Consumo Específico Agua Dulce EDL', 'Valor Base': 210.0, 'Unidad': 'm³/ton LCE', 'Tipo': '%', 'Ayuda': 'Agua dulce requerida por ton LCE (capacidad E_Q_manual). Total anual calculado: Este valor * E_Q_manual.', 'min_mult': 0.25, 'max_mult': 2.0, 'step_mult': 5.0},
    {'Bloque': 'D', 'Parámetro': 'D_Extr_Otros_Usos', 'Descripción': 'Extr. Otros Usos', 'Valor Base': 1.8e7, 'Unidad': 'm³/año', 'Tipo': '%', 'Ayuda': 'Extracción agua dulce otros usos (comunitario, agrícola, etc.). Modelado con función logística.'},

    # --- V (Volumen de Salmuera) ---
    {'Bloque': 'V', 'Parámetro': 'V_inicial_sal', 'Descripción': 'Volumen Inicial V(0) Salmuera', 'Valor Base': 3.5e8, 'Unidad': 'm³', 'Tipo': '%', 'Ayuda': 'Volumen de salmuera explotable inicial estimado.'},
    {'Bloque': 'V', 'Parámetro': 'R_ns_base', 'Descripción': 'Recarga Base Salmuera (R_b)', 'Valor Base': 7e7, 'Unidad': 'm³/año', 'Tipo': '%', 'Ayuda': 'Tasa promedio anual estimada de recarga de salmuera. Alta incertidumbre.'},
    # Parámetros para variación sinusoidal de E_EDL_s_t (la base ahora se calcula internamente)
    {'Bloque': 'V', 'Parámetro': 'V_A_E', 'Descripción': 'Amplitud Extracción Salmuera (A_E)', 'Valor Base': 0.3, 'Unidad': 'fracción (0-1)', 'Tipo': '%', 'Ayuda': 'Variación % sobre tasa base calculada de extracción de salmuera.', 'min_mult': 0.0, 'max_mult': 1.0, 'step_mult': 0.05},
    {'Bloque': 'V', 'Parámetro': 'T_E_V', 'Descripción': 'Periodo Extracción Salmuera (T_E)', 'Valor Base': 1.0, 'Unidad': 'años', 'Tipo': '%', 'Ayuda': 'Duración ciclo de extracción de salmuera.', 'min_mult': 0.1, 'max_mult': 10, 'step_mult': 0.1},
    {'Bloque': 'V', 'Parámetro': 'phi_E_V', 'Descripción': 'Fase Extracción Salmuera (φ_E)', 'Valor Base': 0.5, 'Unidad': 'rad', 'Tipo': '%', 'Ayuda': 'Desfase ciclo extracción de salmuera.', 'min_mult': -math.pi, 'max_mult': math.pi, 'step_mult': 0.1},
    # NUEVOS PARÁMETROS para calcular la tasa base de extracción de salmuera
    {'Bloque': 'V', 'Parámetro': 'L_Planning_Ref_Concentration_base', 'Descripción': 'Conc. Li Planificación (Ext. Salmuera)', 'Valor Base': 600.0, 'Unidad': 'mg/L', 'Tipo': '%', 'Ayuda': 'Conc. Li de referencia para calcular vol. de salmuera a extraer, según E_Q_manual y eficiencia.', 'min_mult': 0.5, 'max_mult': 1.5, 'step_mult': 10.0},
    {'Bloque': 'V', 'Parámetro': 'L_DLE_RecoveryEff_base', 'Descripción': 'Eficiencia Recuperación Li (EDL)', 'Valor Base': 0.85, 'Unidad': 'fracción (0-1)', 'Tipo': '%', 'Ayuda': 'Eficiencia global (0-1) del proceso EDL para obtener LCE de la salmuera procesada.', 'min_mult': 0.5, 'max_mult': 1.1, 'step_mult': 0.01},
    # Resto de parámetros de V (Recarga Salmuera, Evaporación Salmuera)
    {'Bloque': 'V', 'Parámetro': 'A_R', 'Descripción': 'Amplitud Recarga Salmuera (A_R)', 'Valor Base': 0.2, 'Unidad': 'fracción (0-1)', 'Tipo': '%', 'Ayuda': 'Variación % recarga salmuera.', 'min_mult': 0.0, 'max_mult': 1.0, 'step_mult': 0.05},
    {'Bloque': 'V', 'Parámetro': 'T_R_V', 'Descripción': 'Periodo Recarga Salmuera (T_R)', 'Valor Base': 1.0, 'Unidad': 'años', 'Tipo': '%', 'Ayuda': 'Duración ciclo recarga salmuera.', 'min_mult': 0.1, 'max_mult': 10, 'step_mult': 0.1},
    {'Bloque': 'V', 'Parámetro': 'phi_R_V', 'Descripción': 'Fase Recarga Salmuera (φ_R)', 'Valor Base': 0.0, 'Unidad': 'rad', 'Tipo': '%', 'Ayuda': 'Desfase ciclo recarga salmuera.', 'min_mult': -math.pi, 'max_mult': math.pi, 'step_mult': 0.1},
    {'Bloque': 'V', 'Parámetro': 'Ev_s_base', 'Descripción': 'Evaporación Base Salmuera (Ev_b)', 'Valor Base': 4e7, 'Unidad': 'm³/año', 'Tipo': '%', 'Ayuda': 'Evaporación directa de salmuera base.'},
    {'Bloque': 'V', 'Parámetro': 'A_Ev', 'Descripción': 'Amplitud Evap. Salmuera (A_Ev)', 'Valor Base': 0.15, 'Unidad': 'fracción (0-1)', 'Tipo': '%', 'Ayuda': 'Variación % evaporación salmuera.', 'min_mult': 0.0, 'max_mult': 1.0, 'step_mult': 0.05},
    {'Bloque': 'V', 'Parámetro': 'T_Ev_V', 'Descripción': 'Periodo Evap. Salmuera (T_Ev)', 'Valor Base': 1.0, 'Unidad': 'años', 'Tipo': '%', 'Ayuda': 'Duración ciclo evaporación salmuera.', 'min_mult': 0.1, 'max_mult': 10, 'step_mult': 0.1},
    {'Bloque': 'V', 'Parámetro': 'phi_Ev_V', 'Descripción': 'Fase Evap. Salmuera (φ_Ev)', 'Valor Base': 1.0, 'Unidad': 'rad', 'Tipo': '%', 'Ayuda': 'Desfase ciclo evaporación salmuera.', 'min_mult': -math.pi, 'max_mult': math.pi, 'step_mult': 0.1},
    {'Bloque': 'V', 'Parámetro': 'V_scale', 'Descripción': 'Escala Modulación V (V_scl)', 'Valor Base': 3.5e7, 'Unidad': 'm³', 'Tipo': '%', 'Ayuda': 'Umbral para modular salidas de V(t) si V es bajo. (Base: 10% V(0)).'},

    # --- C (Calidad del Agua) ---
    {'Bloque': 'C', 'Parámetro': 'C_inicial', 'Descripción': 'Calidad Inicial C(0)', 'Valor Base': 5.0, 'Unidad': 'mg/L', 'Tipo': '%', 'Ayuda': 'Concentración inicial del indicador de mala calidad C.', 'max_mult': 5.0},
    {'Bloque': 'C', 'Parámetro': 'a', 'Descripción': 'Coef. Ponderación Carga Metales (a)', 'Valor Base': 0.5, 'Unidad': '', 'Tipo': '%', 'Ayuda': 'Pondera la carga total de Metales (fondo + actividad).', 'max_mult': 4.0},
    {'Bloque': 'C', 'Parámetro': 'b', 'Descripción': 'Coef. Ponderación Carga Otros (b)', 'Valor Base': 0.3, 'Unidad': '', 'Tipo': '%', 'Ayuda': 'Pondera la carga total de Otros (fondo + actividad).', 'max_mult': 4.0},
    # M y X ahora son cargas de FONDO
    {'Bloque': 'C', 'Parámetro': 'M_CargaFondo_base', 'Descripción': 'Carga Fondo Metales (M_fondo)', 'Valor Base': 2.0, 'Unidad': 'mg/L/año (en D_ref_C)', 'Tipo': '%', 'Ayuda': 'Tasa base de aporte de contaminantes metálicos de fondo (natural, preexistente).'},
    {'Bloque': 'C', 'Parámetro': 'X_CargaFondo_base', 'Descripción': 'Carga Fondo Otros (X_fondo)', 'Valor Base': 3.0, 'Unidad': 'mg/L/año (en D_ref_C)', 'Tipo': '%', 'Ayuda': 'Tasa base de aporte de "Otros" contaminantes de fondo.'},
    # NUEVOS PARÁMETROS para cargas por actividad EDL
    {'Bloque': 'C', 'Parámetro': 'M_FactorPorProduccion_base', 'Descripción': 'Factor Carga Metales por Prod.', 'Valor Base': 0.025, 'Unidad': '(mg/L/año)/(1kT LCE/año)', 'Tipo': '%', 'Ayuda': 'Carga adicional de Metales por cada 1000 ton LCE/año de E_Q_manual.', 'min_mult': 0.0, 'max_mult': 10.0, 'step_mult': 0.005},
    {'Bloque': 'C', 'Parámetro': 'X_FactorPorProduccion_base', 'Descripción': 'Factor Carga Otros por Prod.', 'Valor Base': 0.040, 'Unidad': '(mg/L/año)/(1kT LCE/año)', 'Tipo': '%', 'Ayuda': 'Carga adicional de Otros contam. por cada 1000 ton LCE/año de E_Q_manual.', 'min_mult': 0.0, 'max_mult': 10.0, 'step_mult': 0.005},
    {'Bloque': 'C', 'Parámetro': 'R_dep', 'Descripción': 'Tasa Depuración (R_dep)', 'Valor Base': 2.0, 'Unidad': 'mg/L/año', 'Tipo': '%', 'Ayuda': 'Tasa constante de eliminación/depuración del contaminante C.'},

    # --- B (Biodiversidad) ---
    {'Bloque': 'B', 'Parámetro': 'B_inicial', 'Descripción': 'Biodiv. Inicial B(0)', 'Valor Base': 100.0, 'Unidad': 'índice', 'Tipo': '%', 'Ayuda': 'Valor inicial del índice de biodiversidad.', 'max_mult': 3.0},
    {'Bloque': 'B', 'Parámetro': 'B_r_base', 'Descripción': 'Tasa Crec. Intrínseco (r)', 'Valor Base': 0.1, 'Unidad': '1/año', 'Tipo': '%', 'Ayuda': 'Tasa crecimiento logístico base de B.', 'min_mult': 0.0, 'max_mult': 1.0, 'step_mult': 0.01},
    {'Bloque': 'B', 'Parámetro': 'B_K_base', 'Descripción': 'Capacidad Carga Base (K_b)', 'Valor Base': 150.0, 'Unidad': 'índice', 'Tipo': '%', 'Ayuda': 'Máximo B sostenible base (antes de estrés D o C).'},
    {'Bloque': 'B', 'Parámetro': 'C_max_stress', 'Descripción': 'C Máx Estrés (B)', 'Valor Base': 150.0, 'Unidad': 'mg/L', 'Tipo': '%', 'Ayuda': 'Nivel de C(t) donde el impacto en K_eff es máximo. (Base fijada).'},
    {'Bloque': 'B', 'Parámetro': 'm', 'Descripción': 'm (NO USADO)', 'Valor Base': 0.0, 'Unidad': 'N/A', 'Tipo': 'Simple', 'Ayuda': 'NO USADO por ode_B_modK.'},
    {'Bloque': 'B', 'Parámetro': 'D_crit', 'Descripción': 'D Crítico (B)', 'Valor Base': 'D(0)*0.5', 'Unidad': 'm³', 'Tipo': 'Simple', 'Ayuda': 'Umbral de D(t) para reducción de K_eff de Biodiversidad (Base: 50% de D(0)).'},
    {'Bloque': 'B', 'Parámetro': 'n', 'Descripción': 'n (NO USADO)', 'Valor Base': 0.0, 'Unidad': 'N/A', 'Tipo': 'Simple', 'Ayuda': 'NO USADO por ode_B_modK.'},
    {'Bloque': 'B', 'Parámetro': 'C_crit', 'Descripción': 'C Crítico (B)', 'Valor Base': 4.0, 'Unidad': 'mg/L', 'Tipo': 'Simple', 'Ayuda': 'Umbral de C(t) para reducción de K_eff de Biodiversidad.'},

    # --- S (Percepción Social) ---
    {'Bloque': 'S', 'Parámetro': 'S_inicial', 'Descripción': 'Percepción Inicial S(0)', 'Valor Base': 50.0, 'Unidad': 'índice', 'Tipo': '%', 'Ayuda': 'Valor inicial del índice de percepción social S.'},
    {'Bloque': 'S', 'Parámetro': 'S_p1', 'Descripción': 'Peso Penaliz. Calidad Agua (p1)', 'Valor Base': 0.05, 'Unidad': 'índice/(mg/L)/año', 'Tipo': '%', 'Ayuda': 'Factor de penalización a S si C(t) > S_C_crit.', 'min_mult': 0.0, 'max_mult': 1.0, 'step_mult': 0.01},
    {'Bloque': 'S', 'Parámetro': 'S_p2', 'Descripción': 'Peso Influencia Biodiversidad (p2)', 'Valor Base': 0.03, 'Unidad': 'índice/índice_B/año', 'Tipo': '%', 'Ayuda': 'Factor de influencia positiva de B(t) sobre S(t).'},
    {'Bloque': 'S', 'Parámetro': 'S_p4', 'Descripción': 'Tasa Decaimiento Percep. (p4)', 'Valor Base': 0.1, 'Unidad': '1/año', 'Tipo': '%', 'Ayuda': 'Tasa de decaimiento natural o "olvido" para el índice S.'},
    {'Bloque': 'S', 'Parámetro': 'S_C_crit_base', 'Descripción': 'Umbral C para Percep. Social (S_C_crit)', 'Valor Base': 5.0, 'Unidad': 'mg/L', 'Tipo': '%', 'Ayuda': 'Umbral de C(t) que activa la penalización social en S(t).', 'min_mult': 0.0, 'max_mult': 100.0},
    # NUEVOS PARÁMETROS para S(t) por D(t)
    {'Bloque': 'S', 'Parámetro': 'S_D_crit_social_base', 'Descripción': 'Umbral Social Crítico Disp. Agua (D)', 'Valor Base': 'D(0)*0.3', 'Unidad': 'm³', 'Tipo': '%', 'Ayuda': 'Volumen D(t) por debajo del cual S(t) se afecta negativamente. Base: 30% D(0).', 'min_mult': 0.1, 'max_mult': 2.0, 'step_mult': 0.05},
    {'Bloque': 'S', 'Parámetro': 'S_p_disp_agua_base', 'Descripción': 'Peso Impacto Escasez Agua en Percep.', 'Valor Base': 0.05, 'Unidad': 'índice/año', 'Tipo': '%', 'Ayuda': 'Magnitud impacto negativo en S(t) por D(t) < S_D_crit_social.', 'min_mult': 0.0, 'max_mult': 0.5, 'step_mult': 0.01},

    # --- L (Concentración de Litio) ---
    {'Bloque': 'L', 'Parámetro': 'L_Aditivos', 'Descripción': 'Aditivos Netos Li (Adi)', 'Valor Base': 25.0, 'Unidad': 'mg/L/año', 'Tipo': '%', 'Ayuda': 'Tasa neta adición/pérdida de conc. Li (procesos no volumétricos).'},
    {'Bloque': 'L', 'Parámetro': 'L_inicial', 'Descripción': 'Litio Inicial L(0)', 'Valor Base': 600.0, 'Unidad': 'mg/L', 'Tipo': '%', 'Ayuda': 'Concentración inicial promedio de Litio en la salmuera.'},
    {'Bloque': 'L', 'Parámetro': 'L_gamma', 'Descripción': 'Coef. Relajación Li (γ)', 'Valor Base': 0.10, 'Unidad': '1/año', 'Tipo': '%', 'Ayuda': 'Tasa de retorno de L(t) hacia L_eq.', 'min_mult': 0.0, 'max_mult': 1.0, 'step_mult': 0.01},
    {'Bloque': 'L', 'Parámetro': 'L_eq', 'Descripción': 'Litio Equilibrio (L_eq)', 'Valor Base': 800.0, 'Unidad': 'mg/L', 'Tipo': '%', 'Ayuda': 'Concentración de Litio de equilibrio/fondo.'},

    # --- E (Beneficios Económicos) ---
    {'Bloque': 'E', 'Parámetro': 'E_P_base', 'Descripción': 'Precio Base Litio (P_base)', 'Valor Base': 10000, 'Unidad': 'u.m./ton LCE', 'Tipo': '%', 'Ayuda': 'Precio venta promedio LCE.'},
    {'Bloque': 'E', 'Parámetro': 'E_A', 'Descripción': 'Amplitud Precio (A)', 'Valor Base': 0.1, 'Unidad': 'fracción (0-1)', 'Tipo': '%', 'Ayuda': 'Variación % del precio base (ciclo).', 'min_mult': 0.0, 'max_mult': 1.0, 'step_mult': 0.01},
    {'Bloque': 'E', 'Parámetro': 'E_T', 'Descripción': 'Periodo Ciclo Precio (T)', 'Valor Base': 1.0, 'Unidad': 'años', 'Tipo': '%', 'Ayuda': 'Duración ciclo precios.', 'min_mult': 0.1, 'max_mult': 10, 'step_mult': 0.1},
    {'Bloque': 'E', 'Parámetro': 'E_phi', 'Descripción': 'Fase Ciclo Precio (φ)', 'Valor Base': 0.0, 'Unidad': 'rad', 'Tipo': '%', 'Ayuda': 'Desfase ciclo precios.', 'min_mult': -math.pi, 'max_mult': math.pi, 'step_mult': 0.1},
    {'Bloque': 'E', 'Parámetro': 'E_C_base', 'Descripción': 'Costo Op. Unitario Base (C_base)', 'Valor Base': 2000, 'Unidad': 'u.m./ton LCE', 'Tipo': '%', 'Ayuda': 'Costo operativo variable unitario si L(t)=E_L_ref.'},
    {'Bloque': 'E', 'Parámetro': 'E_L_ref', 'Descripción': 'Conc. Li Ref. Costo (L_ref)', 'Valor Base': 600, 'Unidad': 'mg/L', 'Tipo': '%', 'Ayuda': 'Conc. Li de referencia para E_C_base.'},
    {'Bloque': 'E', 'Parámetro': 'E_alpha', 'Descripción': 'Exp. Sensib. Costo Op. a L (α)', 'Valor Base': 2.0, 'Unidad': '', 'Tipo': '%', 'Ayuda': 'Sensibilidad del costo op. unitario a cambios en L(t).', 'min_mult': 0, 'max_mult': 5, 'step_mult': 0.1},
    {'Bloque': 'E', 'Parámetro': 'E_Q_manual', 'Descripción': 'Producción Máx Anual LCE (Qmax)', 'Valor Base': 15000, 'Unidad': 'ton LCE/año', 'Tipo': '%', 'Ayuda': 'Capacidad nominal de producción anual. Conduce cálculo de agua y salmuera para EDL.'},
    {'Bloque': 'E', 'Parámetro': 'V_thresh_Q_base', 'Descripción': 'Umbral Vol. Salmuera para Prod. (Vt_Q)', 'Valor Base': 2.8e8, 'Unidad': 'm³', 'Tipo': '%', 'Ayuda': 'Volumen V(t) mínimo para sostener Qmax.'},
    {'Bloque': 'E', 'Parámetro': 'L_thresh_Q_base', 'Descripción': 'Umbral Conc. Li para Prod. (Lt_Q)', 'Valor Base': 400, 'Unidad': 'mg/L', 'Tipo': '%', 'Ayuda': 'Concentración L(t) mínima para operar a Qmax.'},
    # NUEVO PARÁMETRO
    {'Bloque': 'E', 'Parámetro': 'E_CostoFijoOperativoAnual_base', 'Descripción': 'Costo Fijo Operativo Anual', 'Valor Base': 5e6, 'Unidad': 'u.m./año', 'Tipo': '%', 'Ayuda': 'Costos operativos anuales fijos (independientes de producción). VALOR BASE ES ESTIMACIÓN.', 'min_mult': 0.0, 'max_mult': 3.0, 'step_mult': 0.1},
    # NUEVO PARÁMETRO
    {'Bloque': 'E', 'Parámetro': 'E_CAPEX_base', 'Descripción': 'Inversión de Capital Inicial (CAPEX)', 'Valor Base': 500e6, 'Unidad': 'u.m.', 'Tipo': '%', 'Ayuda': 'Costo total de inversión inicial. E(t) iniciará en -CAPEX. VALOR BASE ES ESTIMACIÓN.', 'min_mult': 0.1, 'max_mult': 3.0, 'step_mult': 0.1},
    {'Bloque': 'E', 'Parámetro': 'E_c1_base', 'Descripción': 'Coef. Costo Ext. Calidad Agua (c1)', 'Valor Base': 1e7, 'Unidad': 'u.m./año/(mg/L)', 'Tipo': '%', 'Ayuda': 'Costo anual por externalidad si C(t) > C_crit_E.'},
    {'Bloque': 'E', 'Parámetro': 'C_crit_E_base', 'Descripción': 'Umbral C para Costo Econ. (C_crit_E)', 'Valor Base': 6.0, 'Unidad': 'mg/L', 'Tipo': '%', 'Ayuda': 'Umbral de C(t) para activar costo E_c1.'},
    {'Bloque': 'E', 'Parámetro': 'E_c2_base', 'Descripción': 'Coef. Costo Ext. Biodiversidad (c2)', 'Valor Base': 1e6, 'Unidad': 'u.m./año/índice', 'Tipo': '%', 'Ayuda': 'Costo anual por externalidad si B(t) < B_max_E.'},
    {'Bloque': 'E', 'Parámetro': 'B_max_E_base', 'Descripción': 'Objetivo B para Costo Econ. (B_max_E)', 'Valor Base': 150.0, 'Unidad': 'índice', 'Tipo': '%', 'Ayuda': 'Nivel objetivo de B(t) para costo E_c2.'},
    {'Bloque': 'E', 'Parámetro': 'E_c3_base', 'Descripción': 'Coef. Costo Ext. Social (c3)', 'Valor Base': 1e6, 'Unidad': 'u.m./año/índice', 'Tipo': '%', 'Ayuda': 'Costo anual por externalidad si S(t) < S_max_E.'},
    {'Bloque': 'E', 'Parámetro': 'S_max_E_base', 'Descripción': 'Objetivo S para Costo Econ. (S_max_E)', 'Valor Base': 80.0, 'Unidad': 'índice', 'Tipo': '%', 'Ayuda': 'Nivel objetivo de S(t) para costo E_c3.'},

    # --- IP (Impacto Potencial) --- (Parámetros de definición del indicador, no de la dinámica del sistema)
    {'Bloque': 'IP', 'Parámetro': 'IP_inicial', 'Descripción': 'Impacto Inicial IP(0)', 'Valor Base': 0.0, 'Unidad': 'índice', 'Tipo': '%', 'Ayuda': 'Valor inicial impacto (usualmente 0).', 'min_mult': 0.0, 'max_mult': 1.0},
    {'Bloque': 'IP', 'Parámetro': 'IP_lambda_IP', 'Descripción': 'Tasa Recuperación IP (λ)', 'Valor Base': 0.1, 'Unidad': '1/año', 'Tipo': '%', 'Ayuda': 'Tasa mitigación/decaída IP.', 'min_mult': 0.0, 'max_mult': 2.0},
    {'Bloque': 'IP', 'Parámetro': 'IP_D_ref', 'Descripción': 'Ref. D (p/ IP)', 'Valor Base': 'D(0)', 'Unidad': 'm³', 'Tipo': '%', 'Ayuda': 'Nivel D(t) deseado; si D < Ref, aumenta IP. Base: D(0).'},
    {'Bloque': 'IP', 'Parámetro': 'IP_C_ref', 'Descripción': 'Ref. C (p/ IP)', 'Valor Base': 4.0, 'Unidad': 'mg/L', 'Tipo': '%', 'Ayuda': 'Nivel C(t) máx. aceptable; si C > Ref, aumenta IP.'},
    {'Bloque': 'IP', 'Parámetro': 'IP_B_max', 'Descripción': 'Ref. B (p/ IP)', 'Valor Base': 150.0, 'Unidad': 'índice', 'Tipo': '%', 'Ayuda': 'Nivel B(t) deseado; si B < Ref, aumenta IP.'},
    {'Bloque': 'IP', 'Parámetro': 'IP_S_max', 'Descripción': 'Ref. S (p/ IP)', 'Valor Base': 100.0, 'Unidad': 'índice', 'Tipo': '%', 'Ayuda': 'Nivel S(t) deseado; si S < Ref, aumenta IP.'},
    {'Bloque': 'IP', 'Parámetro': 'IP_E_crit', 'Descripción': 'Ref. E (p/ IP)', 'Valor Base': 1e7, 'Unidad': 'u.m.', 'Tipo': '%', 'Ayuda': 'Beneficio E(t) mínimo aceptable; si E < Ref, aumenta IP.'},
    {'Bloque': 'IP', 'Parámetro': 'IP_V_ref', 'Descripción': 'Ref. V (p/ IP)', 'Valor Base': 'V(0)*0.5', 'Unidad': 'm³', 'Tipo': '%', 'Ayuda': 'Volumen V(t) deseado; si V < Ref, aumenta IP. Base: 50% V(0).'},
    {'Bloque': 'IP', 'Parámetro': 'IP_L_ref_ip', 'Descripción': 'Ref. L (p/ IP)', 'Valor Base': 400.0, 'Unidad': 'mg/L', 'Tipo': '%', 'Ayuda': 'Conc. L(t) deseada; si L < Ref, aumenta IP.'},
    {'Bloque': 'IP', 'Parámetro': 'IP_ip_alpha', 'Descripción': 'Peso D en IP (α)', 'Valor Base': 1.0, 'Unidad': '', 'Tipo': '%', 'Ayuda': 'Importancia desviación D en IP.', 'min_mult': 0.0, 'max_mult': 5.0},
    {'Bloque': 'IP', 'Parámetro': 'IP_ip_beta', 'Descripción': 'Peso C en IP (β)', 'Valor Base': 1.0, 'Unidad': '', 'Tipo': '%', 'Ayuda': 'Importancia desviación C en IP.', 'min_mult': 0.0, 'max_mult': 5.0},
    {'Bloque': 'IP', 'Parámetro': 'IP_ip_gamma', 'Descripción': 'Peso B en IP (γ)', 'Valor Base': 1.0, 'Unidad': '', 'Tipo': '%', 'Ayuda': 'Importancia desviación B en IP.', 'min_mult': 0.0, 'max_mult': 5.0},
    {'Bloque': 'IP', 'Parámetro': 'IP_ip_delta', 'Descripción': 'Peso S en IP (δ)', 'Valor Base': 1.0, 'Unidad': '', 'Tipo': '%', 'Ayuda': 'Importancia desviación S en IP.', 'min_mult': 0.0, 'max_mult': 5.0},
    {'Bloque': 'IP', 'Parámetro': 'IP_ip_epsilon', 'Descripción': 'Peso E en IP (ε)', 'Valor Base': 1.0, 'Unidad': '', 'Tipo': '%', 'Ayuda': 'Importancia desviación E en IP.', 'min_mult': 0.0, 'max_mult': 5.0},
    {'Bloque': 'IP', 'Parámetro': 'IP_ip_zeta', 'Descripción': 'Peso V en IP (ζ)', 'Valor Base': 1.0, 'Unidad': '', 'Tipo': '%', 'Ayuda': 'Importancia desviación V en IP.', 'min_mult': 0.0, 'max_mult': 5.0},
    {'Bloque': 'IP', 'Parámetro': 'IP_ip_eta', 'Descripción': 'Peso L en IP (η)', 'Valor Base': 1.0, 'Unidad': '', 'Tipo': '%', 'Ayuda': 'Importancia desviación L en IP.', 'min_mult': 0.0, 'max_mult': 5.0},

    # --- SH (Seguridad Hídrica) --- (Parámetros de definición del indicador)
    {'Bloque': 'SH', 'Parámetro': 'SH_inicial', 'Descripción': 'Seg. Hídrica Inicial SH(0)', 'Valor Base': 5.0, 'Unidad': 'índice', 'Tipo': '%', 'Ayuda': 'Valor inicial SH.', 'max_mult': 10.0},
    {'Bloque': 'SH', 'Parámetro': 'SH_D_ref', 'Descripción': 'Ref. Disp. Agua (SH)', 'Valor Base': 'D(0)*0.8', 'Unidad': 'm³', 'Tipo': '%', 'Ayuda': 'Nivel D(t) seguro para SH. Base: 80% D(0).'},
    {'Bloque': 'SH', 'Parámetro': 'SH_C_ref', 'Descripción': 'Ref. Calidad Agua (SH)', 'Valor Base': 50.0, 'Unidad': 'mg/L', 'Tipo': '%', 'Ayuda': 'Umbral C(t) máx. aceptable para SH.'},
    {'Bloque': 'SH', 'Parámetro': 'SH_phi', 'Descripción': 'Coef. Generación SH (φ)', 'Valor Base': 1.0, 'Unidad': 'índice/año', 'Tipo': '%', 'Ayuda': 'Factor de aumento potencial de SH.', 'max_mult': 4.0},
    {'Bloque': 'SH', 'Parámetro': 'SH_psi', 'Descripción': 'Tasa Deterioro SH (ψ)', 'Valor Base': 0.5, 'Unidad': '1/año', 'Tipo': '%', 'Ayuda': 'Tasa de decaimiento natural del índice SH. REVISAR VALOR BASE.'}, # Nota para revisión

    # --- FD (Factor de Dilución) --- (Parámetros de definición del indicador)
    {'Bloque': 'FD', 'Parámetro': 'FD_inicial', 'Descripción': 'Factor Dilución Inicial FD(0)', 'Valor Base': 1.0, 'Unidad': 'adim.', 'Tipo': '%', 'Ayuda': 'Valor inicial FD (1 si D(0)=FD_D_ref).', 'min_mult': 0.0},
    {'Bloque': 'FD', 'Parámetro': 'FD_D_ref', 'Descripción': 'Ref. Disp. Agua (FD)', 'Valor Base': 'D(0)', 'Unidad': 'm³', 'Tipo': '%', 'Ayuda': 'Nivel D(t) de referencia para FD. Base: D(0).'},
    {'Bloque': 'FD', 'Parámetro': 'FD_kappa', 'Descripción': 'Constante Respuesta FD (κ)', 'Valor Base': 0.5, 'Unidad': '1/año', 'Tipo': '%', 'Ayuda': 'Velocidad de ajuste de FD a D(t)/FD_D_ref.', 'min_mult': 0.0, 'max_mult': 5.0},
]

# Actualizar las listas de nombres de parámetros para que coincidan con parameter_data_REVISADO
percentage_param_names_full = [p['Parámetro'] for p in parameter_data_REVISADO if p['Tipo'] == '%']
simple_param_names_full = [p['Parámetro'] for p in parameter_data_REVISADO if p['Tipo'] == 'Simple']

# Crear el DataFrame de pandas REVISADO
df_params = pd.DataFrame(parameter_data_REVISADO)

print("\nBloque 6: Definición de parámetros (df_params) REVISADA Y LISTA con todos los cambios conceptuales.")
print(f"Total de parámetros definidos en df_params: {len(df_params)}")
print(f"Parámetros tipo '%': {len(percentage_param_names_full)}, Parámetros tipo 'Simple': {len(simple_param_names_full)}")
print("Los VALORES BASE para los NUEVOS parámetros son iniciales y pueden necesitar ajuste/justificación.")

# Fin del Bloque 6 (REVISADO)


--- Generando Resumen de Parámetros REVISADO (Post-Barrido Conceptual) ---

Bloque 6: Definición de parámetros (df_params) REVISADA Y LISTA con todos los cambios conceptuales.
Total de parámetros definidos en df_params: 93
Parámetros tipo '%': 88, Parámetros tipo 'Simple': 5
Los VALORES BASE para los NUEVOS parámetros son iniciales y pueden necesitar ajuste/justificación.


In [7]:
# ==============================================================================
# Bloque 7: Estructuración de la Interfaz Gráfica (Widgets y Pestañas) - CORREGIDO
# ==============================================================================

# (Asegúrate de que las importaciones de ipywidgets, pandas, math,
# la función create_percentage_widget (Bloque 3), y el DataFrame df_params (Bloque 6 REVISADO)
# ya estén definidas y disponibles en el entorno)

print("\n--- Bloque 7: Configurando Widgets y Pestañas de la Interfaz Gráfica (REVISADO) ---")

# --- Función para obtener el valor base numérico de un parámetro desde df_params ---
# (Esta función ya estaba definida en versiones anteriores del Bloque 7)
def get_base_value(param_name, default=0):
    """
    Obtiene el valor base de un parámetro desde el DataFrame df_params.
    Maneja la conversión a float.
    """
    row = df_params[df_params['Parámetro'] == param_name]
    if not row.empty:
        val = row.iloc[0]['Valor Base']
        # Si el valor es una cadena que indica una dependencia, se devuelve tal cual
        # para ser procesada por la lógica que calcula 'actual_base_val'.
        if isinstance(val, str) and (val.startswith('D(0)') or val.startswith('V(0)')):
            return val # Devolver la cadena para procesamiento posterior
        try:
            return float(val)
        except (ValueError, TypeError):
            return default
    return default

# -- Creación del Widget para el Tiempo de Simulación (Global) --
# (Se asume que all_widgets es un diccionario global inicializado en Bloque 2)
sim_time_from_df = get_base_value('sim_time', 50) # Obtiene valor de df_params

if 'sim_time' not in all_widgets:
    all_widgets['sim_time'] = widgets.FloatSlider(
        value=float(sim_time_from_df), # Asegurar que sea float
        min=5, max=200, step=1,
        description="Tiempo Simulación (años):",
        readout_format='.0f',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='500px')
    )

# -- Crear dinámicamente widgets para TODOS los parámetros definidos en df_params (REVISADO) --
all_param_widgets_list = []

# Parámetros tipo '%'
print("Creando widgets para parámetros de tipo '%'...")
for _, p_info in df_params[df_params['Tipo'] == '%'].iterrows():
    p_name = p_info['Parámetro']
    base_val_str = str(p_info.get('Valor Base', "0")) # Obtener como cadena
    actual_base_val = 0.0

    # --- INICIO DE LÓGICA MEJORADA para calcular actual_base_val ---
    if isinstance(base_val_str, str):
        if base_val_str.startswith('D(0)'):
            factor = 1.0
            if '*' in base_val_str:
                try: factor = float(base_val_str.split('*')[1])
                except (ValueError, IndexError): factor = 1.0
            # Necesitamos el valor numérico de D_V_inicial para este cálculo
            d0_val_for_calc = get_base_value('D_V_inicial', 1e9) # Usa la función get_base_value
            if isinstance(d0_val_for_calc, str): # Si get_base_value devolvió una fórmula para D_V_inicial
                print(f"ADVERTENCIA: D_V_inicial tiene un valor base fórmula '{d0_val_for_calc}'. No se puede calcular '{base_val_str}' para '{p_name}'. Usando 0.")
                actual_base_val = 0.0
            else:
                actual_base_val = d0_val_for_calc * factor
        elif base_val_str.startswith('V(0)'): # NUEVA CONDICIÓN PARA V(0)
            factor = 1.0
            if '*' in base_val_str:
                try: factor = float(base_val_str.split('*')[1])
                except (ValueError, IndexError): factor = 1.0
            # Necesitamos el valor numérico de V_inicial_sal
            v0_sal_for_calc = get_base_value('V_inicial_sal', 3.5e8) # Usa la función get_base_value
            if isinstance(v0_sal_for_calc, str):
                 print(f"ADVERTENCIA: V_inicial_sal tiene un valor base fórmula '{v0_sal_for_calc}'. No se puede calcular '{base_val_str}' para '{p_name}'. Usando 0.")
                 actual_base_val = 0.0
            else:
                actual_base_val = v0_sal_for_calc * factor
        else: # Si no empieza con D(0) o V(0), intentar convertir a float
            try: actual_base_val = float(base_val_str)
            except ValueError:
                print(f"ADVERTENCIA: No se pudo convertir el valor base '{base_val_str}' a float para '{p_name}'. Usando 0.")
                actual_base_val = 0.0
    else: # Si ya es numérico (aunque df_params suele tenerlo como str o num)
        actual_base_val = float(base_val_str) # Asegurar que es float
    # --- FIN DE LÓGICA MEJORADA ---

    widget = create_percentage_widget( # create_percentage_widget está definido en Bloque 3
        name=p_name,
        base_value=actual_base_val,
        description=p_info.get('Descripción', p_name),
        unit=p_info.get('Unidad', ''),
        help_text=p_info.get('Ayuda', ''),
        min_mult=p_info.get('min_mult', 0.0),
        max_mult=p_info.get('max_mult', 2.0),
        step_mult=p_info.get('step_mult', 0.01)
    )
    all_param_widgets_list.append({'name': p_name, 'widget': widget, 'bloque': p_info.get('Bloque', 'Otros')})

# Parámetros tipo 'Simple'
print("Creando widgets para parámetros de tipo 'Simple'...")
for _, p_info in df_params[df_params['Tipo'] == 'Simple'].iterrows():
    p_name = p_info['Parámetro']
    if p_name != 'sim_time': # sim_time ya fue creado
        base_val_str = str(p_info.get('Valor Base', "0"))
        actual_base_val = 0.0

        # --- INICIO DE LÓGICA MEJORADA para calcular actual_base_val (similar a tipo '%') ---
        if isinstance(base_val_str, str):
            if base_val_str.startswith('D(0)'):
                factor = 1.0
                if '*' in base_val_str:
                    try: factor = float(base_val_str.split('*')[1])
                    except (ValueError, IndexError): factor = 1.0
                d0_val_for_calc = get_base_value('D_V_inicial', 1e9)
                if isinstance(d0_val_for_calc, str): actual_base_val = 0.0
                else: actual_base_val = d0_val_for_calc * factor
            elif base_val_str.startswith('V(0)'): # NUEVA CONDICIÓN PARA V(0)
                factor = 1.0
                if '*' in base_val_str:
                    try: factor = float(base_val_str.split('*')[1])
                    except (ValueError, IndexError): factor = 1.0
                v0_sal_for_calc = get_base_value('V_inicial_sal', 3.5e8)
                if isinstance(v0_sal_for_calc, str): actual_base_val = 0.0
                else: actual_base_val = v0_sal_for_calc * factor
            else:
                try: actual_base_val = float(base_val_str)
                except ValueError: actual_base_val = 0.0
        else:
            actual_base_val = float(base_val_str)
        # --- FIN DE LÓGICA MEJORADA ---

        desc_text = p_info.get('Descripción', p_name)
        if p_info.get('Unidad', '') not in ['N/A', '']:
            desc_text = f"{desc_text} [{p_info.get('Unidad', '')}]"

        widget = widgets.FloatText(
            value=actual_base_val,
            description=f"{desc_text}:",
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='400px'),
            tooltip=p_info.get('Ayuda', ''),
            disabled=(p_name in ['m', 'n']) # m y n (de B(t) original) siguen deshabilitados
        )
        all_widgets[p_name] = widget
        all_param_widgets_list.append({'name': p_name, 'widget': widget, 'bloque': p_info.get('Bloque', 'Otros')})

# --- Agrupar widgets por Bloque y crear la Interfaz de Pestañas ---
print("Agrupando widgets en pestañas...")
widgets_by_bloque = {}
for item in all_param_widgets_list:
    bloque = item['bloque']
    if bloque not in widgets_by_bloque:
        widgets_by_bloque[bloque] = []
    # Insertar parámetros 'Simple' al principio de su lista de bloque para orden visual
    # (esto es una preferencia, se puede quitar si se desea orden alfabético o el de df_params)
    if item['name'] in [p['Parámetro'] for p in parameter_data_REVISADO if p['Tipo'] == 'Simple']: # Usa la lista de df_params revisado
        widgets_by_bloque[bloque].insert(0, item['widget'])
    else:
        widgets_by_bloque[bloque].append(item['widget'])

tab_order = ['D', 'V', 'C', 'B', 'S', 'L', 'E', 'IP', 'SH', 'FD', 'Global', 'Otros']
param_tabs_full = widgets.Tab()
tab_children = []
tab_titles = []

for bloque_key in tab_order:
    if bloque_key in widgets_by_bloque and widgets_by_bloque[bloque_key]: # Asegura que hay widgets en el bloque
        widgets_list = [widgets.HTML(f"<b>Parámetros del Bloque: {bloque_key}</b><hr>")] + widgets_by_bloque[bloque_key]
        tab_children.append(widgets.VBox(widgets_list, layout=widgets.Layout(padding="5px")))
        tab_titles.append(f"{bloque_key}")
    elif bloque_key == 'Global' and 'sim_time' in all_widgets:
        widgets_list = [widgets.HTML(f"<b>Parámetros Globales:</b><hr>"), all_widgets['sim_time']]
        tab_children.append(widgets.VBox(widgets_list, layout=widgets.Layout(padding="5px")))
        tab_titles.append(f"{bloque_key}")

param_tabs_full.children = tab_children
for i, title in enumerate(tab_titles):
    param_tabs_full.set_title(i, title)

# --- Área de Salida para los Gráficos de la Simulación (ya definida globalmente o en Bloque 7 original) ---
if 'output_plot_full' not in globals(): # Solo definir si no existe (puede estar en Bloque 2)
    output_plot_full = widgets.Output()

print("Bloque 7: Creación de widgets y pestañas de la interfaz COMPLETADA.")

# Fin del Bloque 7 (CORREGIDO)


--- Bloque 7: Configurando Widgets y Pestañas de la Interfaz Gráfica (REVISADO) ---
Creando widgets para parámetros de tipo '%'...
Creando widgets para parámetros de tipo 'Simple'...
Agrupando widgets en pestañas...
Bloque 7: Creación de widgets y pestañas de la interfaz COMPLETADA.


In [8]:
# ==============================================================================
# Bloque 8: Función Principal de Simulación del Modelo (_core_simulation_full) (VERSIÓN REVISADA)
# ==============================================================================

# (Asegúrate de que las funciones ODE del Bloque 5 y las importaciones necesarias
# como numpy, time, traceback y solve_ivp estén definidas/hechas)

def _core_simulation_full(effective_params):
    """
    Ejecuta la simulación COMPLETA del modelo dinámico (10 variables de estado)
    para un conjunto dado de parámetros efectivos.
    MODIFICADO:
    - E0 (condición inicial de E(t)) ahora es -CAPEX.
    - Asegura que los nuevos parámetros se propaguen a sim_params.
    """
    local_results = {}
    sim_params = {} # Este diccionario contendrá todos los parámetros efectivos para las ODEs

    try:
        # --- 1. Preparar el Diccionario sim_params ---
        # Copia inicial de todos los parámetros efectivos recibidos de la UI/escenario
        sim_params.update(effective_params)

        # --- Calcular y establecer parámetros derivados o que dependen de D_V_inicial ---
        # Esta lógica asegura que las referencias dinámicas se calculen correctamente.
        if 'D_V_inicial' in sim_params:
            d0_val = sim_params['D_V_inicial']
            sim_params['D_ref_evap'] = d0_val
            sim_params['D_parametro_critico'] = d0_val * 0.2 # Usado en extraccion_logistica
            sim_params['D_ref_C'] = d0_val # Usado en ode_C para normalizar carga
            
            # D_crit para Biodiversidad: se obtiene de df_params (puede ser 'D(0)*0.5' o un valor)
            # Si es 'D(0)*0.5', se calcula aquí; si no, ya tiene su valor de effective_params.
            # Esta lógica ya está en run_full_simulation al construir effective_params,
            # pero es bueno asegurar que sim_params tenga el valor numérico final.
            if isinstance(sim_params.get('D_crit'), str) and sim_params.get('D_crit') == 'D(0)*0.5':
                 sim_params['D_crit'] = d0_val * 0.5
            # else: D_crit ya tiene su valor numérico desde effective_params.

            # Similar para S_D_crit_social_base si se definió relativo a D(0)
            if isinstance(sim_params.get('S_D_crit_social_base'), str) and sim_params.get('S_D_crit_social_base') == 'D(0)*0.3':
                sim_params['S_D_crit_social_base'] = d0_val * 0.3

            # Referencias para IP, SH, FD que dependen de D(0) o V(0)
            # Estas también se calculan en run_full_simulation, pero las recalculamos/aseguramos aquí para sim_params
            if isinstance(sim_params.get('IP_D_ref'), str) and sim_params.get('IP_D_ref') == 'D(0)':
                sim_params['IP_D_ref'] = d0_val
            if isinstance(sim_params.get('SH_D_ref'), str) and sim_params.get('SH_D_ref') == 'D(0)*0.8':
                sim_params['SH_D_ref'] = d0_val * 0.8
            if isinstance(sim_params.get('FD_D_ref'), str) and sim_params.get('FD_D_ref') == 'D(0)':
                sim_params['FD_D_ref'] = d0_val
            
            v0_sal_val = sim_params.get('V_inicial_sal', 1e9) # Necesitamos V_inicial_sal para IP_V_ref
            if isinstance(sim_params.get('IP_V_ref'), str) and sim_params.get('IP_V_ref') == 'V(0)*0.5':
                sim_params['IP_V_ref'] = v0_sal_val * 0.5
        else:
            # Fallbacks si D_V_inicial no está (improbable si viene de la UI bien configurada)
            default_d0 = 1e9
            sim_params['D_ref_evap'] = default_d0; sim_params['D_parametro_critico'] = default_d0 * 0.2
            sim_params['D_ref_C'] = default_d0;
            if 'D_crit' not in sim_params: sim_params['D_crit'] = default_d0 * 0.5
            if 'S_D_crit_social_base' not in sim_params: sim_params['S_D_crit_social_base'] = default_d0 * 0.3
            if 'IP_D_ref' not in sim_params: sim_params['IP_D_ref'] = default_d0
            if 'SH_D_ref' not in sim_params: sim_params['SH_D_ref'] = default_d0 * 0.8
            if 'FD_D_ref' not in sim_params: sim_params['FD_D_ref'] = default_d0
            if 'IP_V_ref' not in sim_params: sim_params['IP_V_ref'] = sim_params.get('V_inicial_sal', 1e9) * 0.5


        # --- Asegurar que los parámetros internos para las ODEs usen los nombres clave de df_params ---
        # (Muchos de estos ya se manejan con params.get("NombreCorrecto") dentro de las ODEs/helpers)
        # Por ejemplo, la función get_precipitacion_mensual_vol ya usa params.get('D_area',...)
        # Es importante que los NUEVOS nombres de df_params (Bloque 6 revisado)
        # sean los que se usen en las llamadas params.get(...) en Bloques 4 y 5.
        # Ejemplo: Si M fue renombrado a M_CargaFondo_base en df_params, ode_C debe usar params.get("M_CargaFondo_base").
        # Esto ya lo hemos revisado al modificar los Bloques 4 y 5.

        # --- 2. Condiciones Iniciales para las ODEs ---
        V0_D = sim_params.get('D_V_inicial', 1e11) # Default grande si no está
        V0_V = max(sim_params.get('V_inicial_sal', 0), 0.0)
        C0   = max(sim_params.get('C_inicial', 0), 0.0)
        B0   = max(sim_params.get('B_inicial', 0), 0.0)
        S0   = sim_params.get('S_inicial', 0)
        L0   = max(sim_params.get('L_inicial', 0), 0.0)
        
        # --- MODIFICACIÓN: Condición Inicial para E(t) ---
        CAPEX = sim_params.get("E_CAPEX_base", 0.0) # Obtener el nuevo parámetro CAPEX
        E0   = -CAPEX  # El beneficio económico acumulado inicia en negativo CAPEX.
        # --- FIN DE MODIFICACIÓN ---

        IP0  = max(sim_params.get('IP_inicial', 0), 0.0)
        SH0  = max(sim_params.get('SH_inicial', 0), 0.0)
        FD0  = max(sim_params.get('FD_inicial', 0), 0.0)

        # --- 3. Configuración del Tiempo de Simulación ---
        t_final = sim_params.get('sim_time', 50) # Usar sim_time de sim_params
        num_points = max(300, int(t_final * 10) + 1)
        t_eval = np.linspace(0, t_final, num_points)
        t_span = (0, t_final)

        # --- 4. Ejecutar Simulaciones Secuencialmente ---
        print("Ejecutando ODEs (con nueva lógica)...", end="")
        start_sim_time_ode = time.time() # Renombrado para evitar conflicto con parámetro sim_time

        # 1. D(t)
        sol_D = solve_ivp(ode_D, t_span, [V0_D], args=(sim_params,), t_eval=t_eval, method='RK45', dense_output=True); print(" D", end=".")
        if sol_D.status!=0: raise Exception(f"Fallo D(t): {sol_D.message}")
        local_results['t'] = sol_D.t; local_results['D'] = np.maximum(0, sol_D.sol(local_results['t'])[0])

        # 2. V(t)
        sol_V = solve_ivp(ode_V, t_span, [V0_V], args=(sim_params,), t_eval=t_eval, method='RK45', dense_output=True); print(" V", end=".")
        if sol_V.status!=0: raise Exception(f"Fallo V(t): {sol_V.message}")
        local_results['V'] = np.maximum(0, sol_V.sol(local_results['t'])[0])
        dt_grad = local_results['t'][1] - local_results['t'][0] if len(local_results['t'])>1 else 1 # Renombrado dt
        V_deriv = np.gradient(local_results['V'], dt_grad) if dt_grad > 1e-9 else np.zeros_like(local_results['V'])

        # 3. C(t)
        sol_C = solve_ivp(lambda t,y_C: ode_C(t, y_C, sim_params, local_results['t'], local_results['D']), t_span, [C0], t_eval=t_eval, method='RK45', dense_output=True); print(" C", end=".")
        if sol_C.status!=0: raise Exception(f"Fallo C(t): {sol_C.message}")
        local_results['C'] = np.maximum(0, sol_C.sol(local_results['t'])[0])

        # 4. B(t)
        print(" B(modK)", end=".")
        sol_B = solve_ivp(lambda t,y_B: ode_B_modK(t, y_B, sim_params, local_results['t'], local_results['D'], local_results['C']), t_span, [B0], t_eval=t_eval, method='RK45', dense_output=True);
        if sol_B.status!=0: raise Exception(f"Fallo B(t) [modK]: {sol_B.message}")
        local_results['B'] = np.maximum(0, sol_B.sol(local_results['t'])[0])

        # 5. L(t)
        sol_L = solve_ivp(lambda t,y_L: ode_L(t, y_L, sim_params, local_results['t'], local_results['V'], V_deriv), t_span, [L0], t_eval=t_eval, method='RK45', dense_output=True); print(" L", end=".")
        if sol_L.status!=0: raise Exception(f"Fallo L(t): {sol_L.message}")
        local_results['L'] = np.maximum(0, sol_L.sol(local_results['t'])[0])

        # 6. S(t) - AHORA NECESITA D_arr_para_S
        print(" S(Ref+D)", end=".") # Etiqueta actualizada
        sol_S = solve_ivp(lambda t,y_S: ode_S_refinado(t, y_S, sim_params, local_results['t'], local_results['C'], local_results['B'], local_results['D']), # Pasando D
                          t_span, [S0], t_eval=t_eval, method='RK45', dense_output=True);
        if sol_S.status!=0: raise Exception(f"Fallo S(t): {sol_S.message}")
        local_results['S'] = sol_S.sol(local_results['t'])[0]

        # 7. E(t)
        print(" E(Ref+Fijos+CAPEX)", end=".") # Etiqueta actualizada
        sol_E = solve_ivp(lambda t,y_E: ode_E(t, y_E, sim_params, local_results['t'], local_results['L'], local_results['V'], local_results['C'], local_results['B'], local_results['S']),
                          t_span, [E0], t_eval=t_eval, method='RK45', dense_output=True); # E0 ahora es -CAPEX
        if sol_E.status!=0: raise Exception(f"Fallo E(t): {sol_E.message}")
        local_results['E'] = sol_E.sol(local_results['t'])[0]

        # 8. IP(t)
        global results # Necesario para que dev_X puedan acceder a los resultados actuales
        results_copy_before_ip = results.copy(); results.update(local_results)
        try:
            required_for_dev = ['t', 'D', 'C', 'B', 'S', 'E', 'V', 'L']
            if any(k not in results or results[k] is None for k in required_for_dev): raise ValueError("Faltan resultados previos en 'results' global para calcular IP.")
            sol_IP = solve_ivp(lambda t,y_IP: ode_IP_full(t, y_IP, sim_params), t_span, [IP0], t_eval=t_eval, method='RK45', dense_output=True); print(" IP", end=".")
        finally: results = results_copy_before_ip
        if sol_IP.status!=0: raise Exception(f"Fallo IP(t): {sol_IP.message}")
        local_results['IP'] = np.maximum(0, sol_IP.sol(local_results['t'])[0])

        # 9. SH(t)
        print(" SH(Inv)", end=".")
        sol_SH = solve_ivp(lambda t,y_SH: ode_SH_CalidadInversa(t, y_SH, sim_params, local_results['t'], local_results['D'], local_results['C']), t_span, [SH0], t_eval=t_eval, method='RK45', dense_output=True);
        if sol_SH.status!=0: raise Exception(f"Fallo SH(t): {sol_SH.message}")
        local_results['SH'] = np.maximum(0, sol_SH.sol(local_results['t'])[0])

        # 10. FD(t)
        sol_FD = solve_ivp(lambda t,y_FD: ode_FD(t, y_FD, sim_params, local_results['t'], local_results['D']), t_span, [FD0], t_eval=t_eval, method='RK45', dense_output=True); print(" FD.")
        if sol_FD.status!=0: raise Exception(f"Fallo FD(t): {sol_FD.message}")
        local_results['FD'] = np.maximum(0, sol_FD.sol(local_results['t'])[0])

        local_results['effective_D_V_inicial_used'] = V0_D
        local_results['sim_params_used'] = sim_params.copy() # Importante guardar una copia de los sim_params finales usados
        sim_duration_ode = time.time() - start_sim_time_ode
        print(f"...Simulaciones ODE completadas en {sim_duration_ode:.2f} seg.")
        return local_results

    except Exception as e:
        print(f"\nError en _core_simulation_full: {e}")
        traceback.print_exc()
        t_final_fallback = effective_params.get('sim_time', 50); t_eval_fallback = np.linspace(0, t_final_fallback, 300)
        results_nan = {k: np.full_like(t_eval_fallback, np.nan) for k in ['t','D','V','C','B','S','L','E','IP','SH','FD']}
        results_nan['t'] = t_eval_fallback
        results_nan['effective_D_V_inicial_used'] = np.nan
        results_nan['sim_params_used'] = sim_params.copy() if sim_params else {} # Guarda sim_params si se llegó a crear
        return results_nan

# Fin del Bloque 8 (REVISADO)

In [9]:
# ==============================================================================
# Bloque 9: Función de Ejecución y Visualización (`run_full_simulation`) - COMPLETO Y CORREGIDO
# ==============================================================================

# (Asegúrate de que todas las importaciones necesarias, df_params (Bloque 6 REVISADO),
# all_widgets (poblado por Bloque 7 CORREGIDO), output_plot_full (Bloque 7),
# y _core_simulation_full (Bloque 8 REVISADO) estén definidos previamente)

def run_full_simulation(b): # 'b' es el argumento del evento del botón
    print("Función run_full_simulation INVOCADA por el botón.")
    with output_plot_full:
        clear_output(wait=True)
        print("=============================================")
        print("--- Iniciando Simulación Integrada (Modelo Revisado) ---")
        print("=============================================")
        start_time_total = time.time()

        try:
            # --- 1. Recopilar TODOS los parámetros efectivos y resolver fórmulas ---
            effective_params = {}
            valid_inputs = True

            # Obtener primero los valores base numéricos de D_V_inicial y V_inicial_sal desde sus widgets
            # ya que otros parámetros pueden depender de ellos.
            try:
                d_v_inicial_base_widget = all_widgets.get("D_V_inicial_base")
                d_v_inicial_mult_widget = all_widgets.get("D_V_inicial_multiplier")
                if not (d_v_inicial_base_widget and d_v_inicial_mult_widget):
                    raise ValueError("Widgets para D_V_inicial no encontrados.")
                effective_d0 = d_v_inicial_base_widget.value * d_v_inicial_mult_widget.value
                if not math.isfinite(effective_d0):
                    raise ValueError(f"Valor de D_V_inicial no es finito: {effective_d0}")
                effective_params["D_V_inicial"] = effective_d0 # Guardar el valor numérico
                
                v_inicial_sal_base_widget = all_widgets.get("V_inicial_sal_base")
                v_inicial_sal_mult_widget = all_widgets.get("V_inicial_sal_multiplier")
                if not (v_inicial_sal_base_widget and v_inicial_sal_mult_widget):
                    raise ValueError("Widgets para V_inicial_sal no encontrados.")
                effective_v0_sal = v_inicial_sal_base_widget.value * v_inicial_sal_mult_widget.value
                if not math.isfinite(effective_v0_sal):
                     raise ValueError(f"Valor de V_inicial_sal no es finito: {effective_v0_sal}")
                effective_params["V_inicial_sal"] = effective_v0_sal # Guardar el valor numérico

            except Exception as e_init_param:
                print(f"Error crítico al obtener D_V_inicial o V_inicial_sal: {e_init_param}")
                traceback.print_exc()
                return

            # Listas de nombres de parámetros actualizadas según el df_params del Bloque 6 REVISADO
            # Estas listas se deben generar a partir de df_params para asegurar que son correctas.
            # Por simplicidad, asumimos que df_params está disponible globalmente.
            current_percentage_params = [p['Parámetro'] for _, p in df_params[df_params['Tipo'] == '%'].iterrows() if p['Parámetro'] not in ["D_V_inicial", "V_inicial_sal"]] # Ya procesados
            current_simple_params = [p['Parámetro'] for _, p in df_params[df_params['Tipo'] == 'Simple'].iterrows()]


            print(f"Recopilando parámetros efectivos...")

            # Recopilar parámetros Tipo '%' (excepto los ya obtenidos)
            for name in current_percentage_params:
                base_widget = all_widgets.get(name + "_base")
                multiplier_widget = all_widgets.get(name + "_multiplier")
                if not (base_widget and multiplier_widget):
                    print(f"!!ERROR INTERNO!! Widgets para '{name}' (tipo %) no encontrados.")
                    valid_inputs = False; break
                
                base_val_from_widget = base_widget.value # Este es el valor numérico del widget base
                mult_val = multiplier_widget.value

                if base_val_from_widget is None or not math.isfinite(base_val_from_widget) or \
                   mult_val is None or not math.isfinite(mult_val):
                    print(f"Error en Input (tipo %): Parámetro '{name}'. Valor Base Widget='{base_val_from_widget}', Multiplicador='{mult_val}'")
                    valid_inputs = False; break
                
                # Verificar si el 'Valor Base' original en df_params era una fórmula
                p_info_original = df_params[df_params['Parámetro'] == name].iloc[0]
                original_base_val_str = str(p_info_original.get('Valor Base', "0"))
                
                val_to_use_for_effective_calc = base_val_from_widget # Por defecto, el valor del widget base

                if isinstance(original_base_val_str, str):
                    if original_base_val_str.startswith('D(0)'):
                        factor = 1.0
                        if '*' in original_base_val_str:
                            try: factor = float(original_base_val_str.split('*')[1])
                            except (ValueError, IndexError): factor = 1.0
                        val_to_use_for_effective_calc = effective_d0 * factor
                    elif original_base_val_str.startswith('V(0)'):
                        factor = 1.0
                        if '*' in original_base_val_str:
                            try: factor = float(original_base_val_str.split('*')[1])
                            except (ValueError, IndexError): factor = 1.0
                        val_to_use_for_effective_calc = effective_v0_sal * factor
                
                effective_params[name] = val_to_use_for_effective_calc * mult_val

            if not valid_inputs: print("Simulación cancelada por error en parámetro tipo '%'."); return

            # Recopilar parámetros Tipo 'Simple'
            for name in current_simple_params:
                widget = all_widgets.get(name)
                if not widget:
                    print(f"!!ERROR INTERNO!! Widget para '{name}' (tipo Simple) no encontrado.")
                    valid_inputs = False; break
                
                val_from_widget = widget.value
                
                if val_from_widget is None or (isinstance(val_from_widget, float) and not math.isfinite(val_from_widget)):
                    print(f"Error en Input (tipo Simple): Parámetro '{name}'. Valor='{val_from_widget}'")
                    valid_inputs = False; break

                # Verificar si el 'Valor Base' original en df_params era una fórmula para los Simples
                p_info_original = df_params[df_params['Parámetro'] == name].iloc[0]
                original_base_val_str = str(p_info_original.get('Valor Base', "0"))

                if isinstance(original_base_val_str, str):
                    if original_base_val_str.startswith('D(0)'):
                        factor = 1.0
                        if '*' in original_base_val_str:
                            try: factor = float(original_base_val_str.split('*')[1])
                            except (ValueError, IndexError): factor = 1.0
                        effective_params[name] = effective_d0 * factor
                        widget.value = effective_params[name] # Actualizar el widget si su base era fórmula
                    elif original_base_val_str.startswith('V(0)'):
                        factor = 1.0
                        if '*' in original_base_val_str:
                            try: factor = float(original_base_val_str.split('*')[1])
                            except (ValueError, IndexError): factor = 1.0
                        effective_params[name] = effective_v0_sal * factor
                        widget.value = effective_params[name] # Actualizar el widget
                    else:
                        effective_params[name] = val_from_widget # Usar valor del widget
                else:
                     effective_params[name] = val_from_widget # Usar valor del widget
            
            if not valid_inputs: print("Simulación cancelada por error en parámetro tipo 'Simple'."); return
            
            # Forzar que C_max_stress, C_crit, D_crit (para Biodiversidad) usen los valores de sus widgets simples
            # (ya que _core_simulation_full espera estos nombres directamente en sim_params)
            if 'C_max_stress' in all_widgets: # C_max_stress es ahora tipo '%'
                 effective_params['C_max_stress'] = all_widgets['C_max_stress_base'].value * all_widgets['C_max_stress_multiplier'].value
            if 'C_crit' in all_widgets: effective_params['C_crit'] = all_widgets['C_crit'].value
            if 'D_crit' in all_widgets: effective_params['D_crit'] = all_widgets['D_crit'].value


            print("Recopilación y cálculo de parámetros efectivos completados.")

            # --- 2. Ejecutar Simulación Central ---
            global results; results = {}
            sim_output = _core_simulation_full(effective_params); results.update(sim_output) # Usa Bloque 8 REVISADO
            results['sim_time_used'] = effective_params.get('sim_time'); results['effective_D_V_inicial'] = results.get('effective_D_V_inicial_used')
            results['sim_params_used'] = results.get('sim_params_used', {}); results['model_D'] = "D Revisado"; results['model_V'] = "V Revisado";
            results['model_C'] = "C Revisado"; results['model_B'] = "B ModK"; results['model_S'] = "S Revisado+D"; results['model_L'] = "L (usa E_EDL_s_t rev.)";
            results['model_E'] = "E Revisado (CAPEX+Fijos)"; results['model_IP'] = "IP (refs revisadas)";
            results['model_SH'] = "SH (refs revisadas)"; results['model_FD'] = "FD (rol clarificado)"
            print("--- Simulación Core Finalizada ---")

            # --- 3. Graficar Resultados Seleccionados ---
            print("\n--- Actualizando Gráficos ---")
            t = results.get('t')
            if t is None or len(t) == 0 or np.isnan(results.get('D', np.nan)).all():
                print("ERROR: La simulación falló o no produjo resultados válidos para graficar.")
                fig_err, ax_err = plt.subplots(1, 1, figsize=(10, 2)); ax_err.text(0.5, 0.5, "ERROR EN SIMULACIÓN", color='red', ha='center', va='center', fontsize=16); ax_err.axis('off'); display(fig_err); plt.close(fig_err)
                return

            fig, axes = plt.subplots(3, 4, figsize=(16, 12), sharex=True)
            axes = axes.flatten(); fm = format_param
            plot_config = [
                {'ax_idx': 0, 'key': 'IP', 'color': 'red', 'unit': 'índice', 'label': 'Impacto Potencial'}, {'ax_idx': 1, 'key': 'SH', 'color': 'green', 'unit': 'índice', 'label': 'Seguridad Hídrica'},
                {'ax_idx': 2, 'key': 'FD', 'color': 'blue', 'unit': 'adim.', 'label': 'Factor Dilución'}, {'ax_idx': 3, 'key': 'E', 'color': 'gold', 'unit': 'u.m.', 'label': 'Beneficio Econ. Acum.', 'sci': True}, # Label actualizado
                {'ax_idx': 4, 'key': 'D', 'color': 'cyan', 'unit': 'm³', 'label': 'Disp. Hídrica', 'sci': True}, {'ax_idx': 5, 'key': 'V', 'color': 'magenta', 'unit': 'm³', 'label': 'Vol. Salmuera', 'sci': True},
                {'ax_idx': 6, 'key': 'C', 'color': 'orange', 'unit': 'mg/L', 'label': 'Calidad Agua'}, {'ax_idx': 7, 'key': 'B', 'color': 'purple', 'unit': 'índice', 'label': 'Biodiversidad'},
                {'ax_idx': 8, 'key': 'S', 'color': 'brown', 'unit': 'índice', 'label': 'Percep. Social'}, {'ax_idx': 9, 'key': 'L', 'color': '#8A2BE2', 'unit': 'mg/L', 'label': 'Conc. Litio'},
            ]
            axes[10].set_visible(False); axes[11].set_visible(False)
            for p_cfg in plot_config:
                ax = axes[p_cfg['ax_idx']]; key = p_cfg['key']; data = results.get(key); plot_label = p_cfg['label']
                ax.set_ylabel(f"{key} ({p_cfg['unit']})", fontsize=9); ax.set_title(f'Evolución {plot_label}', fontsize=10); ax.tick_params(axis='both', which='major', labelsize=8); ax.grid(True, alpha=0.6)
                if data is not None:
                    valid_idx = ~np.isnan(data) & (np.isfinite(data))
                    if np.any(valid_idx):
                        t_valid = t[valid_idx]; data_valid = data[valid_idx]
                        if len(t_valid) > 0:
                            ax.plot(t_valid, data_valid, label=f'{key}(t)', color=p_cfg['color'], lw=1.5)
                            if p_cfg.get('sci', False): ax.ticklabel_format(style='sci', axis='y', scilimits=(0,0), useMathText=True)
                            if key == 'FD': ax.axhline(1.0, color='grey', linestyle='--', linewidth=0.8, label='FD Ideal (D=D_ref)')
                            if key == 'E' and "E_CAPEX_base" in effective_params and effective_params["E_CAPEX_base"] > 0 : # Línea de recuperación CAPEX
                                ax.axhline(0.0, color='black', linestyle=':', linewidth=0.8, label='Recuperación CAPEX (E=0)')
                            if key == 'B': k_base_val = effective_params.get('B_K_base', 150); ax.axhline(k_base_val, color='grey', linestyle=':', linewidth=0.8, label=f'K_base={k_base_val:.0f}')
                            if key == 'C': c_crit_val = effective_params.get('C_crit', 4.0); c_max_stress_val = effective_params.get('C_max_stress', 20.0); ax.axhline(c_crit_val, color='red', linestyle=':', linewidth=0.8, label=f'C_crit(B)={c_crit_val:.1f}'); ax.axhline(c_max_stress_val, color='darkred', linestyle=':', linewidth=0.8, label=f'C_max_stress(B)={c_max_stress_val:.1f}')
                            if key == 'D':
                                d_crit_val = effective_params.get('D_crit', np.nan)
                                if math.isfinite(d_crit_val):
                                    ax.axhline(d_crit_val, color='red', linestyle=':', linewidth=0.8, label=f'D_crit(B)={format_param(d_crit_val)}')
                                # También graficar S_D_crit_social_base si está definido
                                s_d_crit_social_val = effective_params.get('S_D_crit_social_base', np.nan)
                                if math.isfinite(s_d_crit_social_val):
                                     ax.axhline(s_d_crit_social_val, color='orange', linestyle='-.', linewidth=0.8, label=f'S_D_crit_soc={format_param(s_d_crit_social_val)}')

                            ax.legend(loc='best', fontsize='x-small')
                        else: ax.text(0.5, 0.5, f'Sin Datos Válidos\npara {key}(t)', ha='center', va='center', transform=ax.transAxes, color='orange', fontsize=9)
                    else: ax.text(0.5, 0.5, f'Sin Datos Válidos\npara {key}(t)', ha='center', va='center', transform=ax.transAxes, color='orange', fontsize=9)
                else: ax.text(0.5, 0.5, f'Error/Sin Datos\npara {key}(t)', ha='center', va='center', transform=ax.transAxes, color='red', fontsize=9)
            last_row_indices = [i for i in range(8, 12) if i < len(axes) and axes[i].get_visible()]
            valid_time = t is not None and len(t) > 0
            for i, ax_ in enumerate(axes):
                if not ax_.get_visible(): continue
                if i not in last_row_indices: ax_.tick_params(labelbottom=False)
                else:
                    ax_.tick_params(labelbottom=True); ax_.set_xlabel('Tiempo (años)', fontsize=9); ax_.xaxis.set_major_locator(MaxNLocator(integer=True, prune='both'))
                    if valid_time: ax_.set_xlim(0, t[-1])
            fig.suptitle("Resultados Simulación Integrada (Modelo Revisado)", fontsize=16, y=1.01) # Título actualizado
            plt.tight_layout(rect=[0, 0.03, 1, 0.98]); display(fig); plt.close(fig)
            print("--- Gráficos Actualizados ---")

            total_time = time.time() - start_time_total
            print(f"\n--- Simulación y Gráficos Completados en {total_time:.1f} seg ---")
            print(f"Para ver los parámetros usados en esta simulación, ejecute la celda del Bloque 11.")

        except KeyError as e:
            print(f"\n--- ERROR DE CLAVE (KeyError) DURANTE LA RECOPILACIÓN/SIMULACIÓN ---")
            print(f"Detalle del error: Falta la clave o parámetro: {e}")
            print("Verifique que el parámetro exista en 'df_params' (Bloque 6 REVISADO),")
            print("que su widget se haya creado/leído correctamente al inicio de 'run_full_simulation',")
            print("o que 'sim_params' en '_core_simulation_full' (Bloque 8 REVISADO) esté bien poblado.")
            traceback.print_exc()
        except Exception as e:
            print(f"\n--- ERROR INESPERADO DURANTE LA SIMULACIÓN/GRAFICACIÓN ---")
            print(f"Tipo de error: {type(e).__name__}, Mensaje: {e}")
            traceback.print_exc()

# Fin del Bloque 9 (CORREGIDO Y REVISADO)

In [10]:
# ==============================================================================
# 4. INTERFAZ Y BOTÓN para Simulación Integrada
#    Ensamblaje final de la interfaz de usuario y el botón de ejecución.
# ==============================================================================

# --- Creación del Botón de Ejecución ---
# Este botón, al ser presionado, llamará a la función run_full_simulation (Bloque 9).
update_button_full = widgets.Button(
    description="Ejecutar Simulación (B: K_eff, C_max_stress=150 Base)", # Texto del botón
    button_style='success', # Estilo del botón (ej. 'primary', 'success', 'info', 'warning', 'danger')
    tooltip='Ejecuta la simulación completa con los parámetros actuales, usando el modelo de Biodiversidad con K_eff y C_max_stress base de 150.', # Ayuda emergente
    icon='play' # Ícono para el botón (de FontAwesome)
)

# --- Vinculación del Botón a la Función de Simulación ---
# Cuando se haga clic en 'update_button_full', se ejecutará la función 'run_full_simulation'.
update_button_full.on_click(run_full_simulation)

# --- Estructura de la Interfaz de Usuario Completa ---
# Se utiliza un VBox (Vertical Box) para apilar los diferentes componentes de la interfaz.
ui_full = widgets.VBox([
    # Título para la sección de parámetros (usando HTML para formato).
    widgets.HTML("<hr><b>Parámetros del Modelo Integrado (B usando K_eff, C_max_stress=150 Base):</b>"),
    # El widget de pestañas que contiene todos los controles de parámetros (creado en Bloque 7).
    param_tabs_full,
    # Título para la sección de resultados.
    widgets.HTML("<hr><b>Resultados de la Simulación:</b>"),
    # El botón para ejecutar la simulación.
    update_button_full,
    # El área de salida donde se mostrarán los gráficos y la tabla de parámetros (creada en Bloque 7).
    output_plot_full
])

# Mensaje final para el usuario, indicando que la interfaz está lista.
print("\nInterfaz lista para Simulación Integrada (v13.1 - C_max_stress=150 Base). Ajusta parámetros y presiona 'Ejecutar Simulación'.")

# --- Mostrar la Interfaz de Usuario Completa ---
# Este es el comando que efectivamente renderiza y muestra la interfaz en la celda del notebook.
try:
    # Descomentar la siguiente línea puede ser útil en algunos entornos de notebook para
    # asegurar que la salida anterior se limpie correctamente antes de mostrar la nueva interfaz.
    # clear_output(wait=True) # Se maneja dentro de run_full_simulation, pero podría ser útil aquí también.

    display(ui_full) # Muestra el VBox que contiene toda la interfaz.

except Exception as display_error:
    # Bloque para manejar errores que puedan ocurrir específicamente durante el `display`.
    # Esto es útil si hay problemas con el entorno de Jupyter o ipywidgets.
    print("\n\n**********************************************************************")
    print(f"ERROR AL INTENTAR MOSTRAR LA INTERFAZ 'ui_full': {display_error}")
    print("Esto puede indicar un problema residual con la creación de widgets o")
    print("con el estado del entorno Jupyter/ipywidgets.")
    print("Intenta reiniciar el kernel y ejecutar este script de nuevo.")
    print("Si el problema persiste, revisa la consola de Javascript del navegador (usualmente F12).")
    print("**********************************************************************\n")
    traceback.print_exc() # Imprime la traza completa del error para ayudar a diagnosticar.

# --- FIN DEL CÓDIGO PYTHON ---
# Fin del Bloque 10


Interfaz lista para Simulación Integrada (v13.1 - C_max_stress=150 Base). Ajusta parámetros y presiona 'Ejecutar Simulación'.


In [11]:
# ==============================================================================
# Bloque 11: Obtener y Visualizar Parámetros de la Última Simulación (DataFrame Personalizado)
# ==============================================================================

# (Asegúrate de que pandas, ipywidgets, display, clear_output,
# traceback, y las variables globales 'results', 'df_params', 'format_param' existan)

print("--- Configurando Bloque 11: Botón para mostrar parámetros de simulación (DataFrame Personalizado) ---")

# Crear un widget de salida específico para la tabla de este bloque
output_table_b11_custom_df = widgets.Output()

# --- Función que se ejecutará al presionar el botón ---
def show_last_sim_params_custom_df_b11(b_event):
    with output_table_b11_custom_df:
        clear_output(wait=True)
        print("Procesando solicitud de parámetros para tabla personalizada...")
        try:
            if 'sim_params_used' in results and results['sim_params_used']:
                params_used_in_sim = results.get('sim_params_used', {})
                
                if not params_used_in_sim:
                    print("No hay datos de parámetros en 'sim_params_used'. Ejecute una simulación primero.")
                    return

                print(f"DEBUG Bloque 11 (Custom DF): Número de items en 'params_used_in_sim': {len(params_used_in_sim)}")
                
                table_data_custom_b11 = []
                # Iterar sobre la definición original de parámetros (df_params)
                # para obtener todas las descripciones, unidades, etc.
                for _, p_info_original in df_params.iterrows():
                    param_name_internal = p_info_original['Parámetro'] # Este es el 'Parametro' (nombre interno)
                    
                    # Obtener el valor que realmente se usó en la simulación
                    valor_usado = params_used_in_sim.get(param_name_internal)

                    if valor_usado is not None:
                        table_data_custom_b11.append({
                            "Bloque Temático": p_info_original['Bloque'],
                            "Parametro": param_name_internal, # Nombre interno/clave del parámetro
                            "Nombre del Parámetro": p_info_original['Descripción'], # Descripción legible
                            "Unidad": p_info_original['Unidad'],
                            "Valor Utilizado": format_param(valor_usado, decimals=6, sci_limit=1e9), # Aumentamos decimales
                            "Detalle Teórico": p_info_original['Ayuda'] # Corresponde al texto de ayuda
                        })
                
                if table_data_custom_b11:
                    df_display_custom_params_b11 = pd.DataFrame(table_data_custom_b11)
                    
                    print("\n--- Parámetros Utilizados en la Última Simulación (Tabla Personalizada) ---")
                    if not df_display_custom_params_b11.empty:
                        # Usar display() directamente sobre el DataFrame
                        # Ajustar opciones de pandas para la visualización en el notebook
                        with pd.option_context('display.max_rows', None,
                                               'display.max_columns', None,
                                               'display.width', 2000, # Ancho más grande para la tabla
                                               'display.colheader_justify', 'left',
                                               'display.max_colwidth', 150): # Aumentar max_colwidth para 'Detalle Teórico'
                            display(df_display_custom_params_b11)
                        print("\nINFO: Tabla de parámetros personalizada mostrada arriba.")
                    else:
                        print("El DataFrame de parámetros personalizados para mostrar está vacío.")
                else:
                    print("No se pudieron recopilar datos para la tabla personalizada ('table_data_custom_b11' está vacía).")
            else:
                print("No se han encontrado resultados de una simulación previa ('sim_params_used' no disponible).")
                print("Por favor, ejecute una simulación usando el botón del Bloque 10 primero.")

        except NameError as ne:
            print(f"ERROR en Bloque 11 (Custom DF): Variable no definida (NameError): {ne}")
            print("Asegúrese de haber ejecutado bloques anteriores que definen 'results', 'df_params', etc.")
        except Exception as e:
            print(f"Ocurrió un error inesperado al generar la tabla de parámetros personalizada: {e}")
            traceback.print_exc()

# --- Creación del Botón y Vinculación ---
btn_show_custom_params_df = widgets.Button(
    description="Ver Parámetros Detallados (Tabla)",
    button_style='warning', # Estilo del botón
    tooltip='Muestra una tabla detallada de los parámetros utilizados en la última simulación.',
    icon='indent'
)
btn_show_custom_params_df.on_click(show_last_sim_params_custom_df_b11)

# --- Mostrar el botón y el área de salida para la tabla ---
print("Presiona el botón para ver la tabla detallada de parámetros de la última simulación.")
display(btn_show_custom_params_df, output_table_b11_custom_df)

# Fin del Bloque 11 (versión DataFrame Personalizado)

--- Configurando Bloque 11: Botón para mostrar parámetros de simulación (DataFrame Personalizado) ---
Presiona el botón para ver la tabla detallada de parámetros de la última simulación.


Button(button_style='warning', description='Ver Parámetros Detallados (Tabla)', icon='indent', style=ButtonSty…

Output()